In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time

plt.rcParams['text.usetex'] = True

# Functions

In [2]:
def demand_name_by_group_index(index):
    list_demand_names = ["Demand (eligible group, 1)", "Demand (eligible group, 2)", \
                         "Demand (ineligible group, 1)", "Demand (ineligible group, 2)", \
                         "Demand (ineligible group, 3)"]
    return list_demand_names[index]

def VoT_name_by_group_index(index):
    list_demand_names = ["VoT (eligible group, 1)", "VoT (eligible group, 2)", \
                         "VoT (ineligible group, 1)", "VoT (ineligible group, 2)", \
                         "VoT (ineligible group, 3)"]
    return list_demand_names[index]

def equals(a, b, tol = 1E-3):
    if abs(a-b) <= tol:
        return True
    else:
        return False
    
def equals_array(arr_1, arr_2, tol = 1E-2):
    if np.linalg.norm(arr_1 - arr_2) <= tol:
        return True
    else:
        return False
    

In [3]:
def latency_max(flow_max, coeff):
    
    assert np.all(coeff >= 0.0), "coeff should be non-negative"
    assert len(coeff.shape) == 1, "coeff should be a 1-D array."
    assert coeff.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
    
    return coeff[0] + max(coeff[1] * (flow_max - coeff[2]), 0)

In [4]:
# arr_1 = np.array([[1, 2, 3], [4, 5, 6]])
# np.linalg.norm(arr_1)

# Download Groups, Routes to Edges Data:

In [5]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_1.csv')

# df_od_flow_data
# df_data

In [6]:
dict_data = {}

for column_name_full in list(df_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_data[column_name_full].tolist()
    else:
        dict_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start City Index" or category == "End City Index":
                dict_data[int(column_name_full)][category] \
                    = int(df_data[column_name_full].tolist()[category_index])
            elif category == "Start City" or category == "End City":
                dict_data[int(column_name_full)][category] \
                    = df_data[column_name_full].tolist()[category_index]
            else:
#                 print("category:", category)
                dict_data[int(column_name_full)][category] \
                    = float(df_data[column_name_full].tolist()[category_index])

# Test git

In [7]:
dict_data

{0: {'Start City Index': 0,
  'End City Index': 0,
  'Start City': 'Palo Alto',
  'End City': 'Palo Alto',
  'O-D Flow (Max Entropy)': 612.539616936298,
  'Demand (eligible group, 1)': 47.77809012103125,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 27.564282762133406,
  'VoT (eligible group, 2)': 0.10238603988603988,
  'Demand (ineligible group, 1)': 115.15744798402402,
  'VoT (ineligible group, 1)': 0.2754407051282051,
  'Demand (ineligible group, 2)': 134.75871572598555,
  'VoT (ineligible group, 2)': 0.5809294871794872,
  'Demand (ineligible group, 3)': 287.28108034312373,
  'VoT (ineligible group, 3)': 1.8596449415012848},
 1: {'Start City Index': 0,
  'End City Index': 1,
  'Start City': 'Palo Alto',
  'End City': 'East Palo Alto',
  'O-D Flow (Max Entropy)': 98.03491986336807,
  'Demand (eligible group, 1)': 7.64672374934271,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 4.411571393851563,
  'VoT (eligible g

In [8]:
cities_dict = {}
for od_info in list(dict_data.values()):
    if od_info["Start City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["Start City Index"]] = od_info["Start City"]
    if od_info["End City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["End City Index"]] = od_info["End City"]

cities_list = list(cities_dict.values())

# cities_dict

In [9]:
od_to_edges_array = np.zeros((len(list(dict_data.keys())), 2))

for od_index, od_info in dict_data.items():
    od_to_edges_array[od_index, 0] = int(cities_list.index(od_info["Start City"]))
    od_to_edges_array[od_index, 1] = int(cities_list.index(od_info["End City"]))

edge_to_od_dict = {}
num_edges = int(np.max(od_to_edges_array)) + 1
# print("num_edges:", num_edges)

for e in range(num_edges):
    edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                           if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]
    
# od_to_edges_array
# edge_to_od_dict

In [10]:
num_groups_per_od = 5

demand_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))
VoT_array_base = np.zeros((len(list(dict_data.keys())), num_groups_per_od))

for od_index, od_value in dict_data.items():
    for group_index in range(num_groups_per_od):
        demand_name = demand_name_by_group_index(group_index)
        VoT_name = VoT_name_by_group_index(group_index)
        
        demand_array[od_index, group_index] = od_value[demand_name]
        VoT_array_base[od_index, group_index] = od_value[VoT_name]

print(demand_array)
# VoT_array_base

[[ 47.77809012  27.56428276 115.15744798 134.75871573 287.28108034]
 [  7.64672375   4.41157139  18.43056493  21.56768237  45.97837742]
 [ 88.76462936  51.21036309 213.94551691 250.3617751  533.7257842 ]
 [ 19.95836002  11.51443847  48.10476517  56.29281031 120.00603652]
 [ 75.66495318  43.65285761 182.37193845 213.41397052 454.95978261]
 [ 18.9161492   10.913163    45.59276988  53.35324135 113.73940996]
 [119.4231777   68.89798713 287.84047959 336.83460377 718.07013258]
 [ 15.23267296  22.71991899  85.716058    74.8724603   59.63978734]
 [  3.42495327   5.10840487  19.27261838  16.83451606  13.40956279]
 [ 12.98400593  19.36597495  73.06254185  63.81969017  50.83568424]
 [  3.2461412    4.84170212  18.26642165  15.95560927  12.70946808]
 [ 20.49356972  30.56668025 115.31974823 100.73110538  80.23753567]
 [  7.61874311   9.23040031  31.06102962  37.36114412  61.2429735 ]
 [ 28.88291825  34.99276635 117.75343596 141.63738759 232.1742275 ]
 [  7.2210114    8.74853304  29.43950801  35.410

In [11]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'

T = 5
VoT_array = np.zeros((VoT_array_base.shape[0], VoT_array_base.shape[1], T))

for t in range(T):
    df_perturbation_data = pd.read_csv(directory_path + 'perturbations_1_' + str(t) + '.csv')
    perturbation_array = df_perturbation_data.to_numpy()[:, 1:]
    VoT_array[:, :, t] = VoT_array_base * perturbation_array
    
# VoT_array_base
# perturbation_array

# Download Latency Parameters Data

In [12]:
directory_path_latency = '../data/pems_latency_inference___101_N_Sep_to_Nov_2024/'
df_latency_params = pd.read_csv(directory_path_latency + 'latency_params.csv')

# list(df_latency_params.loc[:, "Palo Alto"])

In [13]:
dict_latency_params = {}

city_list = list(df_latency_params.columns)[1:]

for city in city_list:
#     if city != "Belmont":
    if 1 == 1:
        dict_latency_params[city] = {}
        dict_latency_params[city]["Flow (at bend)"] = df_latency_params.loc[:, city][0]
        dict_latency_params[city]["Latency (at bend)"] = df_latency_params.loc[:, city][1]
        dict_latency_params[city]["Slope (after bend)"] = df_latency_params.loc[:, city][2]

dict_latency_params

{'Palo Alto': {'Flow (at bend)': 861.9885,
  'Latency (at bend)': 1.326448252,
  'Slope (after bend)': 0.000782666},
 'East Palo Alto': {'Flow (at bend)': 1001.517857,
  'Latency (at bend)': 2.213126553,
  'Slope (after bend)': 0.000584484},
 'Redwood City': {'Flow (at bend)': 881.1846667,
  'Latency (at bend)': 4.892192375,
  'Slope (after bend)': 0.001563724},
 'Belmont': {'Flow (at bend)': 1278.948125,
  'Latency (at bend)': 1.199911179,
  'Slope (after bend)': 0.001994138},
 'San Mateo': {'Flow (at bend)': 1034.092826,
  'Latency (at bend)': 5.541006284,
  'Slope (after bend)': 0.002147262},
 'Burlingame': {'Flow (at bend)': 845.15,
  'Latency (at bend)': 1.503111345,
  'Slope (after bend)': 0.000306601},
 'Millbrae': {'Flow (at bend)': 853.1818182,
  'Latency (at bend)': 2.384328452,
  'Slope (after bend)': 0.000321856}}

# For a fixed case:

In [14]:
# num_edges = 7
num_gp_lanes = 3

el_indices = [0, 1]
in_indices = [2, 3, 4]

num_el = len(el_indices)
num_in = len(in_indices)

coeff_input = np.zeros((3, num_edges))
for counter, city in enumerate(dict_latency_params.keys()):
    coeff_input[0, counter] = dict_latency_params[city]["Latency (at bend)"]
    coeff_input[1, counter] = dict_latency_params[city]["Slope (after bend)"]
    coeff_input[2, counter] = dict_latency_params[city]["Flow (at bend)"]
    

In [15]:
## Set lambdas:

lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

## Initialize tau, alpha values:

filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))

directory_inits = '../data/old___opt_tolls_subsidies_metrics/'
df_inits = pd.read_csv(directory_inits + 'opt_CBCP___' + filename_segment + '.csv')

print("filename_segment:", filename_segment)
print()

inits_tau_arr_as_object = df_inits.to_numpy()[:, 1:6]
inits_B_arr_as_object = df_inits.to_numpy()[0, 7]

argmin_tau = np.zeros((num_edges, T))
argmin_B = 0

for e in range(num_edges):
    for t in range(T):
        argmin_tau[e, t] = inits_tau_arr_as_object[e, t]
        argmin_B = inits_B_arr_as_object

print("argmin_tau:\n", argmin_tau)
print()
print("argmin_B:\n", argmin_B)


filename_segment: 1_1_1

argmin_tau:
 [[0.1  0.19 0.29 0.1  0.04]
 [0.04 0.07 0.21 0.06 0.15]
 [0.79 0.66 0.98 0.07 0.79]
 [0.54 0.8  0.43 0.24 0.3 ]
 [0.74 0.46 1.2  1.   0.39]
 [0.04 0.13 0.1  0.07 0.08]
 [0.15 0.14 0.11 0.1  0.14]]

argmin_B:
 3.65


In [16]:
# argmin_tau = np.array([[0.0, 0.3194, 0.3194, 0.0, 0.0], \
#                        [0.2498, 0.2498, 0.2498, 0.0, 0.0], \
#                        [0.0, 0.0, 0.0, 0.9995, 0.9995], \
#                        [0.0, 1.0281, 1.0281, 1.0281, 0.0], \
#                        [1.6043, 0.0, 0.0, 1.6043, 0.0], \
#                        [0.0, 0.1922, 0.1922, 0.1922, 0.0], \
#                        [0.0, 0.0, 0.0, 0.2178, 0.2178]])

# argmin_B = 10.6925

##  <font color='blue'>Import argmin_y directly (requires storing it from the other Jupyter notebook file).</font> 

In [17]:
# argmin_y = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
#                                     argmin_tau, argmin_B, od_to_edges_array, \
#                                     demand_array, VoT_array, num_el, coeff_input)

# print("argmin_tau:", argmin_tau)
# print("argmin_B:", argmin_B)
# # print("min_welfare:", min_welfare)
# print()
# print("argmin_y:\n")

# argmin_y


In [18]:
## Set lambdas:

lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

directory_path = '../data/opt_values/'
filename = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I)) + '___y.csv'

y_CBCP_opt_array = pd.read_csv(directory_path + '1_1_1___y_CBCP.csv').to_numpy()

y_CBCP_opt = {}
for row_index in range(y_CBCP_opt_array.shape[0]):
    key = tuple(y_CBCP_opt_array[row_index,:5].astype(int))
    y_CBCP_opt[key] = y_CBCP_opt_array[row_index,-1]

# y_CBCP_array
y_CBCP_opt


{(0, 0, 0, 0, 0): 47.77719854,
 (0, 0, 0, 1, 0): 0.000891184,
 (0, 1, 0, 0, 0): 27.56339627,
 (0, 1, 0, 1, 0): 0.000886267,
 (0, 2, 0, 0, 0): 0.000135711,
 (0, 2, 0, 1, 0): 115.1573123,
 (0, 3, 0, 0, 0): 0.000424958,
 (0, 3, 0, 1, 0): 134.7582908,
 (0, 4, 0, 0, 0): 0.015837252,
 (0, 4, 0, 1, 0): 287.2652431,
 (0, 0, 0, 0, 1): 47.77762321,
 (0, 0, 0, 1, 1): 0.000466512,
 (0, 1, 0, 0, 1): 27.56381774,
 (0, 1, 0, 1, 1): 0.000464798,
 (0, 2, 0, 0, 1): 0.000102942,
 (0, 2, 0, 1, 1): 115.157345,
 (0, 3, 0, 0, 1): 0.000191431,
 (0, 3, 0, 1, 1): 134.7585243,
 (0, 4, 0, 0, 1): 0.007355432,
 (0, 4, 0, 1, 1): 287.2737249,
 (0, 0, 0, 0, 2): 47.77778199,
 (0, 0, 0, 1, 2): 0.000307736,
 (0, 1, 0, 0, 2): 27.56397594,
 (0, 1, 0, 1, 2): 0.000306591,
 (0, 2, 0, 0, 2): 6.53e-05,
 (0, 2, 0, 1, 2): 115.1573826,
 (0, 3, 0, 0, 2): 0.000116053,
 (0, 3, 0, 1, 2): 134.7585997,
 (0, 4, 0, 0, 2): 0.006869666,
 (0, 4, 0, 1, 2): 287.2742107,
 (0, 0, 0, 0, 3): 47.77718778,
 (0, 0, 0, 1, 3): 0.000901948,
 (0, 1, 0, 0

In [19]:
# arr = np.array([11, 22, 33, 44, 55])
# tuple(arr)

# Compute Aggregated Flows, Travel Times, and Cost Metrics

In [20]:
def compute_y_agg_x(y, edge_to_od_dict, num_edges, T):
    
    y_agg = {}
    x = {}
    
    for e in range(num_edges):
        for k in [0, 1]:
            for t in range(T):
                y_agg[(e, k, t, "in")] = sum(y[(od, g, e, k, t)] \
                                                for od in edge_to_od_dict[e] for g in in_indices)
                y_agg[(e, k, t, "el")] = sum(y[(od, g, e, k, t)] \
                                                for od in edge_to_od_dict[e] for g in el_indices)

                x[(e, k, t)] = sum(y[(od, g, e, k, t)] \
                                        for od in edge_to_od_dict[e] for g in in_indices + el_indices)

    return y_agg, x


def compute_travel_times(y_agg, x, edge_to_od_dict, coeff_input, num_gp_lanes, num_edges, T):
    
    travel_times = {}
    avg_travel_times = {}
    
    for e in range(num_edges):
        for t in range(T):
            travel_times[(e, 0, t)] = coeff_input[0, e] \
                + coeff_input[1, e] * max(x[(e, 0, t)] - coeff_input[2, e], 0.0)
            travel_times[(e, 1, t)] = coeff_input[0, e] \
                + coeff_input[1, e] * max(x[(e, 1, t)]/num_gp_lanes - coeff_input[2, e], 0.0)
    
    for e in range(num_edges):
        avg_travel_times[(e, 'ex')] = sum([travel_times[(e, 0, t)] for t in range(T)]) / T
        avg_travel_times[(e, 'gp')] = sum([travel_times[(e, 1, t)] for t in range(T)]) / T
        
        avg_travel_times[(e, 'el')] = sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                        / sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T))
        avg_travel_times[(e, 'in')] = sum(y_agg[(e, k, t, 'in')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                        / sum(y_agg[(e, k, t, 'in')] for k in range(2) for t in range(T))
    
    avg_travel_times['ex'] = sum([travel_times[(e, 0, t)] for e in range(num_edges) for t in range(T)]) / T
    avg_travel_times['gp'] = sum([travel_times[(e, 1, t)] for e in range(num_edges) for t in range(T)]) / T

    avg_travel_times['el'] = sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for e in range(num_edges) for k in range(2) for t in range(T)) \
                                / sum(y_agg[(e, k, t, 'el')] for e in range(num_edges) for k in range(2) for t in range(T))
    avg_travel_times['in'] = sum(y_agg[(e, k, t, 'in')] * travel_times[(e, k, t)] for e in range(num_edges) for k in range(2) for t in range(T)) \
                                / sum(y_agg[(e, k, t, 'in')] for e in range(num_edges) for k in range(2) for t in range(T))
    
    return travel_times, avg_travel_times

def compute_percent_on_express(y_agg, x, num_edges, T):
    
    percent_on_express = {}
    
    for e in range(num_edges):
        percent_on_express[(e, 'el')] = sum(y_agg[(e, 0, t, 'el')] for t in range(T)) \
                                        / sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T))
        percent_on_express[(e, 'in')] = sum(y_agg[(e, 0, t, 'in')] for t in range(T)) \
                                        / sum(y_agg[(e, k, t, 'in')] for k in range(2) for t in range(T))
        percent_on_express[(e, 'all')] = sum(y_agg[(e, 0, t, 'el')] + y_agg[(e, 0, t, 'in')] for t in range(T)) \
                                        / sum(y_agg[(e, k, t, 'el')] + y_agg[(e, k, t, 'in')] for k in range(2) for t in range(T))
    
    percent_on_express['el'] = sum(y_agg[(e, 0, t, 'el')] for e in range(num_edges) for t in range(T)) \
                                    / sum(y_agg[(e, k, t, 'el')] for e in range(num_edges) for k in range(2) for t in range(T))
    percent_on_express['in'] = sum(y_agg[(e, 0, t, 'in')] for e in range(num_edges) for t in range(T)) \
                                    / sum(y_agg[(e, k, t, 'in')] for e in range(num_edges) for k in range(2) for t in range(T))
    percent_on_express['all'] = sum(y_agg[(e, 0, t, 'el')] + y_agg[(e, 0, t, 'in')] for e in range(num_edges) for t in range(T)) \
                                    / sum(y_agg[(e, k, t, 'el')] + y_agg[(e, k, t, 'in')] for e in range(num_edges) for k in range(2) for t in range(T))
    
    return percent_on_express

# def compute_obj_CBCP_together(y, x, travel_times, VoT_array, tau, edge_to_od_dict, lambda_list, num_edges, T):
#     obj = {}
    
#     lambda_E, lambda_R, lambda_I = lambda_list
    
#     for e in range(num_edges):
#         obj[(e, 'E')] = sum( y[(od, g, e, k, t)] * VoT_array[od, g, t] * travel_times[(e, k, t)] \
#                             for od in edge_to_od_dict[e] for g in el_indices for k in range(2) for t in range(T) )

#         obj[(e, 'R')] = sum( y[(od, g, e, 0, t)] * tau[e, t] \
#                         for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

#         obj[(e, 'I')] = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[(e, 0, t)] + tau[e, t]) \
#                             for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
#                         + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[(e, 1, t)] \
#                             for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

#         obj[e] = lambda_E * obj[(e, 'E')] - lambda_R * obj[(e, 'R')] + lambda_I * obj[(e, 'I')]  
    
#     obj['total'] = sum([obj[e] for e in range(num_edges)])
    
#     return obj


def compute_obj_CBCP(y, x, travel_times, VoT_array, tau, edge_to_od_dict, lambda_list, num_edges, T):
    obj = {}
    
    lambda_E, lambda_R, lambda_I = lambda_list
    
    for e in range(num_edges):
        for t in range(T):
            obj[(e, t, 'E')] = sum( y[(od, g, e, k, t)] * VoT_array[od, g, t] * travel_times[(e, k, t)] \
                                   for od in edge_to_od_dict[e] for g in el_indices for k in range(2))

            obj[(e, t, 'R')] = sum( y[(od, g, e, 0, t)] * tau[e, t] \
                                   for od in edge_to_od_dict[e] for g in in_indices)
            
            obj[(e, t, 'I')] = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[(e, 0, t)] + tau[e, t]) \
                                   for od in edge_to_od_dict[e] for g in in_indices) \
                                + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[(e, 1, t)] \
                                      for od in edge_to_od_dict[e] for g in in_indices)

            obj[(e, t)] = lambda_E * obj[(e, t, 'E')] - lambda_R * obj[(e, t, 'R')] + lambda_I * obj[(e, t, 'I')]  
        
        obj[(e, 'E')] = sum([obj[(e, t, 'E')] for t in range(T)])
        obj[(e, 'I')] = sum([obj[(e, t, 'I')] for t in range(T)])
        obj[(e, 'R')] = sum([obj[(e, t, 'R')] for t in range(T)])
        
        obj[e] = lambda_E * obj[(e, 'E')] - lambda_R * obj[(e, 'R')] + lambda_I * obj[(e, 'I')]  
    
    obj['total'] = sum([obj[e] for e in range(num_edges)])
    
    return obj


In [21]:
y_agg_CBCP_opt, x_CBCP_opt \
    = compute_y_agg_x(y = y_CBCP_opt, \
                      edge_to_od_dict = edge_to_od_dict,\
                      num_edges = num_edges, \
                      T = T)

travel_times_CBCP_opt, avg_travel_times_CBCP_opt \
    = compute_travel_times(y_agg = y_agg_CBCP_opt, \
                           x = x_CBCP_opt, \
                           edge_to_od_dict = edge_to_od_dict,\
                           coeff_input = coeff_input, \
                           num_gp_lanes = num_gp_lanes,\
                           num_edges = num_edges, \
                           T = T)

percent_on_express_CBCP_opt \
    = compute_percent_on_express(y_agg = y_agg_CBCP_opt,\
                                 x = x_CBCP_opt, \
                                 num_edges = num_edges, \
                                 T = T)

lambda_list = [lambda_E, lambda_R, lambda_I]

# obj_CBCP_opt_together \
#     = compute_obj_CBCP_together(y = y_CBCP_opt, \
#                        x = x_CBCP_opt, \
#                        travel_times = travel_times_CBCP_opt, \
#                        VoT_array = VoT_array, \
#                        tau = argmin_tau, \
#                        edge_to_od_dict = edge_to_od_dict, \
#                        lambda_list = lambda_list, \
#                        num_edges = num_edges, \
#                        T = T)

obj_CBCP_opt \
    = compute_obj_CBCP(y = y_CBCP_opt, \
                       x = x_CBCP_opt, \
                       travel_times = travel_times_CBCP_opt, \
                       VoT_array = VoT_array, \
                       tau = argmin_tau, \
                       edge_to_od_dict = edge_to_od_dict, \
                       lambda_list = lambda_list, \
                       num_edges = num_edges, \
                       T = T)

In [22]:
# for e in range(num_edges):
#     print("Edge:", e)
#     assert abs(obj_CBCP_opt_together[(e, 'E')] - obj_CBCP_opt[(e, 'E')]) <= 1E-3
#     assert abs(obj_CBCP_opt_together[(e, 'R')] - obj_CBCP_opt[(e, 'R')]) <= 1E-3
#     assert abs(obj_CBCP_opt_together[(e, 'I')] - obj_CBCP_opt[(e, 'I')]) <= 1E-3

In [23]:
# y_CBCP_opt

In [24]:
# y_agg_CBCP_opt
# x_CBCP_opt

# travel_times_CBCP_opt
# avg_travel_times_CBCP_opt
# percent_on_express_CBCP
obj_CBCP_opt


{(0, 0, 'E'): 54.68527513431109,
 (0, 0, 'R'): 88.4089798158,
 (0, 0, 'I'): 8197.047944775291,
 (0, 0): 8163.3242400938025,
 (0, 1, 'E'): 52.53092292145674,
 (0, 1, 'R'): 101.43163421923,
 (0, 1, 'I'): 7995.679946171526,
 (0, 1): 7946.779234873753,
 (0, 2, 'E'): 53.085202034709155,
 (0, 2, 'R'): 201.36624812882002,
 (0, 2, 'I'): 8113.405968483635,
 (0, 2): 7965.124922389524,
 (0, 3, 'E'): 54.853532320387814,
 (0, 3, 'R'): 93.2498003457,
 (0, 3, 'I'): 8133.439505665538,
 (0, 3): 8095.043237640225,
 (0, 4, 'E'): 55.17052148460877,
 (0, 4, 'R'): 44.19934203935999,
 (0, 4, 'I'): 8423.334904331901,
 (0, 4): 8434.30608377715,
 (0, 'E'): 270.3254538954736,
 (0, 'I'): 40862.90826942789,
 (0, 'R'): 528.65600454891,
 0: 40604.57771877445,
 (1, 0, 'E'): 95.76299701489351,
 (1, 0, 'R'): 35.36595711599999,
 (1, 0, 'I'): 11942.89230448277,
 (1, 0): 12003.289344381663,
 (1, 1, 'E'): 94.89661182335578,
 (1, 1, 'R'): 41.06518859576001,
 (1, 1, 'I'): 11551.859180574635,
 (1, 1): 11605.69060380223,
 (1, 

In [25]:
## For reference, from above:

# compute_y_agg_x_CBCP(y, edge_to_od_dict, num_edges, T)
# compute_travel_times_CBCP(y_agg, x, edge_to_od_dict, coeff_input, num_gp_lanes, num_edges, T)
# compute_percent_on_express_CBCP(y_agg, x, num_edges, T)
# compute_obj_CBCP(y, x, travel_times, VoT_array, tau, edge_to_od_dict, lambda_list, num_edges, T)


## Store into opt data array:

In [26]:
opt_data_array = np.zeros((num_edges, 16))

# argmin_tau
opt_data_array[:, 0:5] = argmin_tau

# argmin_tau_avg
opt_data_array[:, 5] = np.mean(argmin_tau, axis=1)

# argmin_B (as array)
opt_data_array[:, 6] = argmin_B * np.eye(1, num_edges, 0)

# percent_on_express (overall)
# percent_on_express (eligible)
# percent_on_express (ineligible)
opt_data_array[:, 7] = np.array([percent_on_express_CBCP_opt[e, 'all'] for e in range(num_edges)]) * 100
opt_data_array[:, 8] = np.array([percent_on_express_CBCP_opt[e, 'el'] for e in range(num_edges)]) * 100
opt_data_array[:, 9] = np.array([percent_on_express_CBCP_opt[e, 'in'] for e in range(num_edges)]) * 100

# avg_travel_time (express lane)
# avg_travel_time (general purpose lane)
opt_data_array[:, 10] = np.array([avg_travel_times_CBCP_opt[(e, 'ex')] for e in range(num_edges)])
opt_data_array[:, 11] = np.array([avg_travel_times_CBCP_opt[(e, 'gp')] for e in range(num_edges)])

# obj_E = {}
# obj_I = {}
# obj_R = {}
# obj
opt_data_array[:, 12] = np.array([obj_CBCP_opt[(e, 'E')] for e in range(num_edges)]) 
opt_data_array[:, 13] = np.array([obj_CBCP_opt[(e, 'I')] for e in range(num_edges)]) 
opt_data_array[:, 14] = np.array([obj_CBCP_opt[(e, 'R')] for e in range(num_edges)]) 
opt_data_array[:, 15] = np.array([obj_CBCP_opt[e] for e in range(num_edges)]) 


opt_data_array = np.round(opt_data_array, decimals=2)

In [27]:
column_names = []
column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
column_names += ["tau (time-averaged)", \
                 "B", \
                 "% overall users using express lanes", \
                 "% eligible users using express lanes", \
                 "% ineligible users using express lanes", \
                 "Average travel time (express lanes)", \
                 "Average travel time (general purpose lanes)", \
                 "Total travel cost (eligible users)", \
                 "Total travel cost (ineligible users)", \
                 "Total toll revenue", \
                 "Total societal cost"]

row_names = ["e=" + str(k+1) for k in range(num_edges) ]

df_opt_save = pd.DataFrame(opt_data_array, index=row_names, columns=column_names)

df_opt_save

,tau (t=1),tau (t=2),tau (t=3),tau (t=4),tau (t=5),tau (time-averaged),B,% overall users using express lanes,% eligible users using express lanes,% ineligible users using express lanes,Average travel time (express lanes),Average travel time (general purpose lanes),Total travel cost (eligible users),Total travel cost (ineligible users),Total toll revenue,Total societal cost
e=1,0.10,0.19,0.29,0.10,0.04,0.14,3.65,23.45,51.45,19.52,1.54,1.62,270.33,40862.91,528.66,40604.58
e=2,0.04,0.07,0.21,0.06,0.15,0.11,0.00,23.58,52.89,19.30,2.34,2.40,477.31,59567.53,456.36,59588.47
e=3,0.79,0.66,0.98,0.07,0.79,0.66,0.00,22.50,32.46,21.09,5.90,6.26,1654.17,202572.45,3956.81,200269.80
e=4,0.54,0.80,0.43,0.24,0.30,0.46,0.00,23.44,41.22,20.96,1.48,1.73,383.66,49246.41,2310.31,47319.76
e=5,0.74,0.46,1.20,1.00,0.39,0.76,0.00,23.18,42.79,20.51,7.27,7.69,2341.75,283040.83,5077.85,280304.73
e=6,0.04,0.13,0.10,0.07,0.08,0.08,0.00,23.30,45.27,20.45,1.72,1.76,438.04,54933.45,477.45,54894.04
e=7,0.15,0.14,0.11,0.10,0.14,0.13,0.00,22.69,60.13,17.84,2.63,2.70,723.34,89396.66,732.08,89387.91


In [28]:
# random_string = ""
# for idx_rand_str in range(10):
#     random_string += str(np.random.randint(1, 9))

# directory_to_save = "../data/old___opt_tolls_subsidies_metrics/"
# random_filename = "opt_CBCP_params___" + random_string + '.csv'

# df_opt_save.to_csv(directory_to_save + random_filename)

In [29]:
# random_string

## Initialize DBCP flows

In [30]:
# Function for transforming arrays of size (n, ) into arrays of size (n, 1)

def pad_dim(arr_or_list):
    arr = np.array(arr_or_list)
    assert len(arr.shape) == 1, "We must have len(arr.shape) == 1 to proceed"
    arr_len = arr.shape[0]
    return arr.reshape((arr_len, 1))

# Function for filling a vector from the bottom up to some value:

def fill_from_bottom(arr_or_list, val):
    arr = np.array(arr_or_list)
    assert np.all(arr >= -1E-3), "We must have all entries of arr >= 0.0."
    assert val >= -1E-3, "We must have val >= 0.0."
    
    arr_fill_from_bottom = np.zeros(arr.shape)
    
    index_boundary = max([index for index in range(arr.shape[0]) if np.sum(arr[index:]) >= val - 1E-3])
    
    for index in range(arr.shape[0]):
        if index > index_boundary:
            arr_fill_from_bottom[index] = arr[index]
        elif index < index_boundary:
            arr_fill_from_bottom[index] = 0.0
        else:
            arr_fill_from_bottom[index] = arr[index_boundary] - (np.sum(arr[index_boundary:]) - val)
        assert arr_fill_from_bottom[index] >= 0.0, "We must have arr_fill_from_bottom[index] >= 0.0"

    assert abs(np.sum(arr_fill_from_bottom) - val) <= 1E-3, \
        "We must have np.sum(arr_fill_from_bottom) == val"
    
    return arr_fill_from_bottom, index_boundary


In [31]:
# dict_VoTs_demands_annotated = {}

# directory_path = "../data/VoTs_demands_sorted/"

# e, t = 0, 0
# filename_in = str(e) + "_" + str(t) + "_" + "in.csv"
# df_data_in = pd.read_csv(directory_path + filename_in)
# dict_VoTs_demands_annotated[(e, t, "in")] = df_data_in.to_numpy()

# dict_VoTs_demands_annotated[(e, t, "in")]

In [32]:
start_time = time.time()

dict_VoTs_demands_annotated = {}

directory_path = "../data/VoTs_demands_sorted/"

for e in range(num_edges):
    for t in range(T):
        
        # Ineligible users:
        
        filename_in = str(e) + "_" + str(t) + "_" + "in.csv"
        df_data_in = pd.read_csv(directory_path + filename_in)
        dict_VoTs_demands_annotated[(e, t, "in")] = df_data_in.to_numpy()
        
        # Eligible users:
        
        filename_el = str(e) + "_" + str(t) + "_" + "el.csv"
        df_data_el = pd.read_csv(directory_path + filename_el)
        dict_VoTs_demands_annotated[(e, t, "el")] = df_data_el.to_numpy()

end_time = time.time()
print("Time:", end_time - start_time)

Time: 0.20371413230895996


In [33]:
# dict_VoTs_demands_annotated

In [34]:
# def y_agg_to_y_DBCP_full(y_agg, dict_VoTs_demands_annotated, num_edges, T):
    
#     y_DBCP = {}
#     VoT_in_boundary = np.zeros((num_edges, T))
#     VoT_el_boundary = np.zeros((num_edges, T))
#     alpha = np.zeros((num_edges, T))

#     for e in range(num_edges):
#         for t in range(T):

#             ## Ineligible user flows:

#             VoTs_demands_annotated_in = dict_VoTs_demands_annotated[(e, t, "in")]        
#             demands_annotated_in_ex_DBCP, index_VoT_in_boundary \
#                 = fill_from_bottom(VoTs_demands_annotated_in[:, 1], y_agg[(e, 0, t, 'in')])
#             demands_annotated_in_gp_DBCP \
#                 = VoTs_demands_annotated_in[:, 1] - demands_annotated_in_ex_DBCP

#             assert np.all(demands_annotated_in_ex_DBCP) >= 0.0, "We must have demands_annotated_in_ex_DBCP >= 0.0"
#             assert np.all(demands_annotated_in_gp_DBCP) >= 0.0, "We must have demands_annotated_in_gp_DBCP >= 0.0"

#             for row_index in range(VoTs_demands_annotated_in.shape[0]):
#                 od, g = VoTs_demands_annotated_in[row_index, 2:]
#                 od, g = int(od), int(g)
#                 y_DBCP[(od, g, e, 0, t)] = demands_annotated_in_ex_DBCP[row_index]
#                 y_DBCP[(od, g, e, 1, t)] = demands_annotated_in_gp_DBCP[row_index]

#             ## Eligible user flows:

#             VoTs_demands_annotated_el = dict_VoTs_demands_annotated[(e, t, "el")]        
#             demands_annotated_el_ex_DBCP, index_VoT_el_boundary \
#                 = fill_from_bottom(VoTs_demands_annotated_el[:, 1], y_agg[(e, 0, t, 'el')])
#             demands_annotated_el_gp_DBCP \
#                 = VoTs_demands_annotated_el[:, 1] - demands_annotated_el_ex_DBCP

#             assert np.all(demands_annotated_el_ex_DBCP) >= 0.0, "We must have demands_annotated_el_ex_DBCP >= 0.0"
#             assert np.all(demands_annotated_el_gp_DBCP) >= 0.0, "We must have demands_annotated_el_gp_DBCP >= 0.0"

#             for row_index in range(VoTs_demands_annotated_el.shape[0]):
#                 od, g = VoTs_demands_annotated_el[row_index, 2:]
#                 od, g = int(od), int(g)
#                 y_DBCP[(od, g, e, 0, t)] = demands_annotated_el_ex_DBCP[row_index]
#                 y_DBCP[(od, g, e, 1, t)] = demands_annotated_el_gp_DBCP[row_index]
                
#             ## Compute alpha:
        
#             VoT_in_boundary[e, t] = VoTs_demands_annotated_in[index_VoT_in_boundary, 0]
#             VoT_el_boundary[e, t] = VoTs_demands_annotated_el[index_VoT_el_boundary, 0]
#             alpha[e, t] = 1 - VoT_el_boundary[e, t] / VoT_in_boundary[e, t]
        
#             assert 0.0 <= alpha[e, t] <= 1.0, "We must have 0.0 <= alpha[e, t] <= 1.0."
    
#     return y_DBCP, VoT_in_boundary, VoT_el_boundary, alpha



In [35]:
# y_DBCP_init, VoT_in_boundary, VoT_el_boundary, init_alpha \
#     = y_agg_to_y_DBCP(y_agg = y_agg_CBCP_opt, \
#                       dict_VoTs_demands_annotated = dict_VoTs_demands_annotated, \
#                       num_edges = num_edges, \
#                       T = T)

In [36]:
# # dict_1 = {'a': 1, 'b': 2}
# dict_1 = {}
# dict_2 = {'c': 11, 'd': 22}

# dict_1 = {**dict_1, **dict_2} 
# dict_1

In [37]:
def y_agg_to_y_DBCP_at_e_t(y_agg_in_at_e_t, y_agg_el_at_e_t, \
                           VoTs_demands_annotated_in, VoTs_demands_annotated_el, \
                           e, t):
    
    y_DBCP_at_e_t = {}
    
    ## Ineligible user flows:

#     VoTs_demands_annotated_in = dict_VoTs_demands_annotated[(e, t, "in")]        
    demands_annotated_in_ex_DBCP, index_VoT_in_boundary \
        = fill_from_bottom(VoTs_demands_annotated_in[:, 1], y_agg_in_at_e_t)
    demands_annotated_in_gp_DBCP \
        = VoTs_demands_annotated_in[:, 1] - demands_annotated_in_ex_DBCP

    assert np.all(demands_annotated_in_ex_DBCP) >= 0.0, "We must have demands_annotated_in_ex_DBCP >= 0.0"
    assert np.all(demands_annotated_in_gp_DBCP) >= 0.0, "We must have demands_annotated_in_gp_DBCP >= 0.0"

    for row_index in range(VoTs_demands_annotated_in.shape[0]):
        od, g = VoTs_demands_annotated_in[row_index, 2:]
        od, g = int(od), int(g)
        y_DBCP_at_e_t[(od, g, e, 0, t)] = demands_annotated_in_ex_DBCP[row_index]
        y_DBCP_at_e_t[(od, g, e, 1, t)] = demands_annotated_in_gp_DBCP[row_index]

    ## Eligible user flows:

#     VoTs_demands_annotated_el = dict_VoTs_demands_annotated[(e, t, "el")]        
    demands_annotated_el_ex_DBCP, index_VoT_el_boundary \
        = fill_from_bottom(VoTs_demands_annotated_el[:, 1], y_agg_el_at_e_t)
    demands_annotated_el_gp_DBCP \
        = VoTs_demands_annotated_el[:, 1] - demands_annotated_el_ex_DBCP

    assert np.all(demands_annotated_el_ex_DBCP) >= 0.0, "We must have demands_annotated_el_ex_DBCP >= 0.0"
    assert np.all(demands_annotated_el_gp_DBCP) >= 0.0, "We must have demands_annotated_el_gp_DBCP >= 0.0"

    for row_index in range(VoTs_demands_annotated_el.shape[0]):
        od, g = VoTs_demands_annotated_el[row_index, 2:]
        od, g = int(od), int(g)
        y_DBCP_at_e_t[(od, g, e, 0, t)] = demands_annotated_el_ex_DBCP[row_index]
        y_DBCP_at_e_t[(od, g, e, 1, t)] = demands_annotated_el_gp_DBCP[row_index]

    ## Compute alpha:

    VoT_in_boundary = VoTs_demands_annotated_in[index_VoT_in_boundary, 0]
    VoT_el_boundary = VoTs_demands_annotated_el[index_VoT_el_boundary, 0]
    alpha = 1 - VoT_el_boundary / VoT_in_boundary

    assert 0.0 <= alpha <= 1.0, "We must have 0.0 <= alpha <= 1.0."
    
    return y_DBCP_at_e_t, VoT_in_boundary, VoT_el_boundary, alpha


In [38]:
# y_DBCP_init___from_func = {}
# init_alpha___from_func = np.zeros((num_edges, T))
# VoT_in_boundary___from_func = np.zeros((num_edges, T))
# VoT_el_boundary___from_func = np.zeros((num_edges, T))

# for e in range(num_edges):
#     for t in range(T):
        
#         y_DBCP_init_at_e_t, VoT_in_boundary___from_func[e, t], VoT_el_boundary___from_func[e, t], init_alpha___from_func[e, t] \
#             = y_agg_to_y_DBCP_at_e_t(y_agg_in_at_e_t = y_agg_CBCP_opt[(e, 0, t, 'in')], \
#                                      y_agg_el_at_e_t = y_agg_CBCP_opt[(e, 0, t, 'el')], \
#                                      VoTs_demands_annotated_in = dict_VoTs_demands_annotated[(e, t, "in")], \
#                                      VoTs_demands_annotated_el = dict_VoTs_demands_annotated[(e, t, "el")], \
#                                      e = e, \
#                                      t = t)
#         y_DBCP_init___from_func = {**y_DBCP_init___from_func, **y_DBCP_init_at_e_t}
        
#         assert 0.0 <= init_alpha___from_func[e, t] <= 1.0, "We must have 0.0 <= init_alpha___from_func[e, t] <= 1.0."
        
#         # y_agg_CBCP_opt[(e, 0, t, 'el')]


In [39]:
y_DBCP_init = {}
init_alpha = np.zeros((num_edges, T))
VoT_in_boundary = np.zeros((num_edges, T))
VoT_el_boundary = np.zeros((num_edges, T))

for e in range(num_edges):
    for t in range(T):
        
        y_DBCP_init_at_e_t, VoT_in_boundary[e, t], VoT_el_boundary[e, t], init_alpha[e, t] \
            = y_agg_to_y_DBCP_at_e_t(y_agg_in_at_e_t = y_agg_CBCP_opt[(e, 0, t, 'in')], \
                                     y_agg_el_at_e_t = y_agg_CBCP_opt[(e, 0, t, 'el')], \
                                     VoTs_demands_annotated_in = dict_VoTs_demands_annotated[(e, t, "in")], \
                                     VoTs_demands_annotated_el = dict_VoTs_demands_annotated[(e, t, "el")], \
                                     e = e, \
                                     t = t)
        y_DBCP_init = {**y_DBCP_init, **y_DBCP_init_at_e_t}
        
        assert 0.0 <= init_alpha[e, t] <= 1.0, "We must have 0.0 <= init_alpha[e, t] <= 1.0."
        
        # y_agg_CBCP_opt[(e, 0, t, 'el')]


In [40]:
# y_DBCP_init = {}
# init_alpha = np.zeros((num_edges, T))
# VoT_in_boundary = np.zeros((num_edges, T))
# VoT_el_boundary = np.zeros((num_edges, T))

# for e in range(num_edges):
#     for t in range(T):
        
#         ## Ineligible user flows:
        
#         VoTs_demands_annotated_in = dict_VoTs_demands_annotated[(e, t, "in")]        
#         demands_annotated_in_ex_DBCP, index_VoT_in_boundary \
#             = fill_from_bottom(VoTs_demands_annotated_in[:, 1], y_agg_CBCP_opt[(e, 0, t, 'in')])
#         demands_annotated_in_gp_DBCP \
#             = VoTs_demands_annotated_in[:, 1] - demands_annotated_in_ex_DBCP
        
#         assert np.all(demands_annotated_in_ex_DBCP) >= 0.0, "We must have demands_annotated_in_ex_DBCP >= 0.0"
#         assert np.all(demands_annotated_in_gp_DBCP) >= 0.0, "We must have demands_annotated_in_gp_DBCP >= 0.0"
        
#         for row_index in range(VoTs_demands_annotated_in.shape[0]):
#             od, g = VoTs_demands_annotated_in[row_index, 2:]
#             od, g = int(od), int(g)
#             y_DBCP_init[(od, g, e, 0, t)] = demands_annotated_in_ex_DBCP[row_index]
#             y_DBCP_init[(od, g, e, 1, t)] = demands_annotated_in_gp_DBCP[row_index]
        
#         ## Eligible user flows:
        
#         VoTs_demands_annotated_el = dict_VoTs_demands_annotated[(e, t, "el")]        
#         demands_annotated_el_ex_DBCP, index_VoT_el_boundary \
#             = fill_from_bottom(VoTs_demands_annotated_el[:, 1], y_agg_CBCP_opt[(e, 0, t, 'el')])
#         demands_annotated_el_gp_DBCP \
#             = VoTs_demands_annotated_el[:, 1] - demands_annotated_el_ex_DBCP
        
#         assert np.all(demands_annotated_el_ex_DBCP) >= 0.0, "We must have demands_annotated_el_ex_DBCP >= 0.0"
#         assert np.all(demands_annotated_el_gp_DBCP) >= 0.0, "We must have demands_annotated_el_gp_DBCP >= 0.0"
        
#         for row_index in range(VoTs_demands_annotated_el.shape[0]):
#             od, g = VoTs_demands_annotated_el[row_index, 2:]
#             od, g = int(od), int(g)
#             y_DBCP_init[(od, g, e, 0, t)] = demands_annotated_el_ex_DBCP[row_index]
#             y_DBCP_init[(od, g, e, 1, t)] = demands_annotated_el_gp_DBCP[row_index]
            
#         ## Compute alpha:
        
#         VoT_in_boundary[e, t] = VoTs_demands_annotated_in[index_VoT_in_boundary, 0]
#         VoT_el_boundary[e, t] = VoTs_demands_annotated_el[index_VoT_el_boundary, 0]
#         init_alpha[e, t] = 1 - VoT_el_boundary[e, t] / VoT_in_boundary[e, t]
        
#         assert 0.0 <= init_alpha[e, t] <= 1.0, "We must have 0.0 <= init_alpha[e, t] <= 1.0."
        
#         # y_agg_CBCP_opt[(e, 0, t, 'el')]
    

In [41]:
# # y_DBCP_init___from_func
# # init_alpha___from_func
# # VoT_in_boundary___from_func
# # VoT_el_boundary___from_func

# assert y_DBCP_init___from_func.keys() == y_DBCP_init.keys()

# for key in y_DBCP_init.keys():
#     assert equals(y_DBCP_init___from_func[key], y_DBCP_init[key])
    
# assert equals_array(init_alpha___from_func, init_alpha)
# assert equals_array(VoT_in_boundary___from_func, VoT_in_boundary)
# assert equals_array(VoT_el_boundary___from_func, VoT_el_boundary)



In [42]:


# # y_DBCP___from_func, VoT_in_boundary___from_func, VoT_el_boundary___from_func, alpha___from_func

# assert y_DBCP_init___from_func.keys() == y_DBCP_init.keys()

# for key in y_DBCP_init___from_func.keys():
#     print("key:", key)
#     assert equals(y_DBCP_init___from_func[key], y_DBCP_init[key])

# for e in range(num_edges):
#     for t in range(T):
#         print()
#         print("e:", e)
#         print("t:", t)
#         print()
        
#         assert equals(VoT_in_boundary___from_func[e, t], VoT_in_boundary[e, t])
#         assert equals(VoT_el_boundary___from_func[e, t], VoT_el_boundary[e, t])
#         assert equals(init_alpha___from_func[e, t], init_alpha[e, t])

# # equals_array()

In [43]:
# list(init_DBCP_flows.keys())
# list(y_CBCP_opt.keys())

# y_DBCP_init

In [44]:
# demand_array.shape
# VoT_array.shape

In [45]:
# # TODO: Test the following definitions in the for loops to record boundary flow, VoT and demand information:

def y_boundary_toggle_setup(y, demand_array, VoT_array, el_indices, in_indices, \
                            VoT_el_boundary, VoT_in_boundary, num_edges, T):
        
    y_agg_boundary_to_toggle = {}
    y_agg_el_ex_low = np.zeros((num_edges, T))
    y_agg_el_ex_high = np.zeros((num_edges, T))

    y_agg_el_at_boundary_VoT = np.zeros((num_edges, T))
    y_agg_in_at_boundary_VoT = np.zeros((num_edges, T))
    demand_el_at_boundary_VoT = np.zeros((num_edges, T))
    demand_in_at_boundary_VoT = np.zeros((num_edges, T))
    y_agg_el_above_boundary_VoT = np.zeros((num_edges, T))
    y_agg_in_above_boundary_VoT = np.zeros((num_edges, T))
    y_agg_at_boundary_VoT = np.zeros((num_edges, T))

    for e in range(num_edges):
        for t in range(T):
    #         print()
    #         print("e:", e)
    #         print("t:", t)
    #         print()

            y_agg_el_at_boundary_VoT[e, t] \
                = sum([y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in el_indices \
                       if abs(VoT_array[od, g, t] - VoT_el_boundary[e, t]) <= 1E-3])
            y_agg_in_at_boundary_VoT[e, t] \
                = sum([y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in in_indices \
                       if abs(VoT_array[od, g, t] - VoT_in_boundary[e, t]) <= 1E-3])
            y_agg_at_boundary_VoT[e, t] = y_agg_el_at_boundary_VoT[e, t] + y_agg_in_at_boundary_VoT[e, t]

    #         y_agg_at_boundary_VoT[e, t] \
    #             = sum([y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in el_indices \
    #                    if abs(VoT_array[od, g, t] - VoT_el_boundary[e, t]) <= 1E-3]) \
    #                 + sum([y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in in_indices \
    #                        if abs(VoT_array[od, g, t] - VoT_in_boundary[e, t]) <= 1E-3])

            demand_el_at_boundary_VoT[e, t] \
                = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in el_indices \
                       if abs(VoT_array[od, g, t] - VoT_el_boundary[e, t]) <= 1E-3])
            demand_in_at_boundary_VoT[e, t] \
                = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices \
                       if abs(VoT_array[od, g, t] - VoT_in_boundary[e, t]) <= 1E-3])

            if abs(y_agg_el_at_boundary_VoT[e, t] - demand_el_at_boundary_VoT[e, t]) <= 1E-3 and \
                    abs(y_agg_in_at_boundary_VoT[e, t] - demand_in_at_boundary_VoT[e, t]) <= 1E-3:
                y_agg_boundary_to_toggle[(e, t)] = False
            else:
                y_agg_boundary_to_toggle[(e, t)] = True

            y_agg_el_above_boundary_VoT[e, t] \
                = sum([y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in el_indices \
                       if VoT_array[od, g, t] > VoT_el_boundary[e, t] + 1E-3])

            y_agg_in_above_boundary_VoT[e, t] \
                = sum([y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in in_indices \
                       if VoT_array[od, g, t] > VoT_in_boundary[e, t] + 1E-3])

            demand_el_above_boundary_VoT \
                = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in el_indices \
                       if VoT_array[od, g, t] > VoT_el_boundary[e, t] + 1E-3])
            demand_in_above_boundary_VoT \
                = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices \
                       if VoT_array[od, g, t] > VoT_in_boundary[e, t] + 1E-3])

            # print("y_agg_el_above_boundary_VoT[e, t]:", y_agg_el_above_boundary_VoT[e, t])
            # print("demand_el_above_boundary_VoT:", demand_el_above_boundary_VoT)
            # print("y_agg_in_above_boundary_VoT[e, t]:", y_agg_in_above_boundary_VoT[e, t])
            # print("demand_in_above_boundary_VoT:", demand_in_above_boundary_VoT)

            y_agg_el_ex_low[e, t] = demand_el_above_boundary_VoT \
                + max(y_agg_at_boundary_VoT[e, t] - demand_in_at_boundary_VoT[e, t], 0.0)

            y_agg_el_ex_high[e, t] = demand_el_above_boundary_VoT \
                + min(y_agg_at_boundary_VoT[e, t], demand_el_at_boundary_VoT[e, t])

            assert y_agg_el_ex_low[e, t] <= y_agg_el_ex_high[e, t] + 1E-3, \
                "We must have y_agg_el_ex_low[e, t] <= y_agg_el_ex_high[e, t]."

            assert abs(y_agg_el_above_boundary_VoT[e, t] - demand_el_above_boundary_VoT) <= 1E-3, \
                "We should have y_agg_el_above_boundary_VoT[e, t] = demand_el_above_boundary_VoT"

            assert abs(y_agg_in_above_boundary_VoT[e, t] - demand_in_above_boundary_VoT) <= 1E-3, \
                "We should have y_agg_in_above_boundary_VoT[e, t] = demand_in_above_boundary_VoT"
    
    return y_agg_boundary_to_toggle, y_agg_el_ex_low, y_agg_el_ex_high


In [46]:
# # Old code which executes the same functionalities as the above function definition:

# y_agg_boundary_to_toggle = {}
# y_agg_el_ex_low = np.zeros((num_edges, T))
# y_agg_el_ex_high = np.zeros((num_edges, T))

# y_agg_el_at_boundary_VoT = np.zeros((num_edges, T))
# y_agg_in_at_boundary_VoT = np.zeros((num_edges, T))
# demand_el_at_boundary_VoT = np.zeros((num_edges, T))
# demand_in_at_boundary_VoT = np.zeros((num_edges, T))
# y_agg_el_above_boundary_VoT = np.zeros((num_edges, T))
# y_agg_in_above_boundary_VoT = np.zeros((num_edges, T))
# y_agg_at_boundary_VoT = np.zeros((num_edges, T))

# for e in range(num_edges):
#     for t in range(T):
# #         print()
# #         print("e:", e)
# #         print("t:", t)
# #         print()

#         y_agg_el_at_boundary_VoT[e, t] \
#             = sum([y_DBCP_init[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in el_indices \
#                    if abs(VoT_array[od, g, t] - VoT_el_boundary[e, t]) <= 1E-3])
#         y_agg_in_at_boundary_VoT[e, t] \
#             = sum([y_DBCP_init[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in in_indices \
#                    if abs(VoT_array[od, g, t] - VoT_in_boundary[e, t]) <= 1E-3])
#         y_agg_at_boundary_VoT[e, t] = y_agg_el_at_boundary_VoT[e, t] + y_agg_in_at_boundary_VoT[e, t]
        
# #         y_agg_at_boundary_VoT[e, t] \
# #             = sum([y_DBCP_init[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in el_indices \
# #                    if abs(VoT_array[od, g, t] - VoT_el_boundary[e, t]) <= 1E-3]) \
# #                 + sum([y_DBCP_init[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in in_indices \
# #                        if abs(VoT_array[od, g, t] - VoT_in_boundary[e, t]) <= 1E-3])
        
#         demand_el_at_boundary_VoT[e, t] \
#             = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in el_indices \
#                    if abs(VoT_array[od, g, t] - VoT_el_boundary[e, t]) <= 1E-3])
#         demand_in_at_boundary_VoT[e, t] \
#             = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices \
#                    if abs(VoT_array[od, g, t] - VoT_in_boundary[e, t]) <= 1E-3])
        
#         if abs(y_agg_el_at_boundary_VoT[e, t] - demand_el_at_boundary_VoT[e, t]) <= 1E-3 and \
#                 abs(y_agg_in_at_boundary_VoT[e, t] - demand_in_at_boundary_VoT[e, t]) <= 1E-3:
#             y_agg_boundary_to_toggle[(e, t)] = False
#         else:
#             y_agg_boundary_to_toggle[(e, t)] = True
        
#         y_agg_el_above_boundary_VoT[e, t] \
#             = sum([y_DBCP_init[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in el_indices \
#                    if VoT_array[od, g, t] > VoT_el_boundary[e, t] + 1E-3])
        
#         y_agg_in_above_boundary_VoT[e, t] \
#             = sum([y_DBCP_init[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in in_indices \
#                    if VoT_array[od, g, t] > VoT_in_boundary[e, t] + 1E-3])
        
#         demand_el_above_boundary_VoT \
#             = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in el_indices \
#                    if VoT_array[od, g, t] > VoT_el_boundary[e, t] + 1E-3])
#         demand_in_above_boundary_VoT \
#             = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices \
#                    if VoT_array[od, g, t] > VoT_in_boundary[e, t] + 1E-3])
        
#         # print("y_agg_el_above_boundary_VoT[e, t]:", y_agg_el_above_boundary_VoT[e, t])
#         # print("demand_el_above_boundary_VoT:", demand_el_above_boundary_VoT)
#         # print("y_agg_in_above_boundary_VoT[e, t]:", y_agg_in_above_boundary_VoT[e, t])
#         # print("demand_in_above_boundary_VoT:", demand_in_above_boundary_VoT)
        
#         y_agg_el_ex_low[e, t] = demand_el_above_boundary_VoT \
#             + max(y_agg_at_boundary_VoT[e, t] - demand_in_at_boundary_VoT[e, t], 0.0)

#         y_agg_el_ex_high[e, t] = demand_el_above_boundary_VoT \
#             + min(y_agg_at_boundary_VoT[e, t], demand_el_at_boundary_VoT[e, t])
        
#         assert y_agg_el_ex_low[e, t] <= y_agg_el_ex_high[e, t] + 1E-3, \
#             "We must have y_agg_el_ex_low[e, t] <= y_agg_el_ex_high[e, t]."
        
#         assert abs(y_agg_el_above_boundary_VoT[e, t] - demand_el_above_boundary_VoT) <= 1E-3, \
#             "We should have y_agg_el_above_boundary_VoT[e, t] = demand_el_above_boundary_VoT"
        
#         assert abs(y_agg_in_above_boundary_VoT[e, t] - demand_in_above_boundary_VoT) <= 1E-3, \
#             "We should have y_agg_in_above_boundary_VoT[e, t] = demand_in_above_boundary_VoT"
        

In [47]:
y_agg_boundary_to_toggle, y_agg_el_ex_low, y_agg_el_ex_high \
    = y_boundary_toggle_setup(y = y_DBCP_init, \
                                 demand_array = demand_array, \
                                 VoT_array = VoT_array, \
                                 el_indices = el_indices, \
                                 in_indices = in_indices, \
                                 VoT_el_boundary = VoT_el_boundary, \
                                 VoT_in_boundary = VoT_in_boundary, \
                                 num_edges = num_edges, \
                                 T = T)

# for e in range(num_edges):
#     for t in range(T):
#         print()
#         print("e:", e)
#         print("t:", t)
#         print()
        
#         assert y_agg_boundary_to_toggle___from_func == y_agg_boundary_to_toggle
#         assert equals(y_agg_el_ex_low___from_func[e, t], y_agg_el_ex_low[e, t])
#         assert equals(y_agg_el_ex_high___from_func[e, t], y_agg_el_ex_high[e, t])


# y_agg_boundary_el_ex_high - y_agg_boundary_el_ex_low


In [48]:
# y_agg_boundary_to_toggle

## Check if DBCP initialization outperforms CBCP

In [49]:
# def compute_obj_DBCP_together(y, x, travel_times, VoT_array, tau, alpha, edge_to_od_dict, lambda_list, num_edges, T):
#     obj = {}
    
#     lambda_E, lambda_R, lambda_I = lambda_list
    
#     for e in range(num_edges):
#         obj[(e, 'E')] = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[(e, 0, t)] + (1 - alpha[e, t]) * tau[e, t]) \
#                             for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
#                         + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[(e, 1, t)] \
#                             for od in edge_to_od_dict[e] for g in el_indices for t in range(T) )

#         obj[(e, 'R')] = sum( y[(od, g, e, 0, t)] * (1 - alpha[e, t]) * tau[e, t] \
#                             for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
#                         + sum( y[(od, g, e, 0, t)] * tau[e, t] \
#                             for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

#         obj[(e, 'I')] = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[(e, 0, t)] + tau[e, t]) \
#                             for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
#                         + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[(e, 1, t)] \
#                             for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

#         obj[e] = lambda_E * obj[(e, 'E')] - lambda_R * obj[(e, 'R')] + lambda_I * obj[(e, 'I')]  
    
#     obj['total'] = sum([obj[e] for e in range(num_edges)])
    
#     return obj


def compute_obj_DBCP(y, x, travel_times, VoT_array, tau, alpha, edge_to_od_dict, lambda_list, num_edges, T):
    obj = {}
    
    lambda_E, lambda_R, lambda_I = lambda_list
    
    for e in range(num_edges):
        for t in range(T):
            obj[(e, t, 'E')] = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[(e, 0, t)] + (1 - alpha[e, t]) * tau[e, t]) \
                                for od in edge_to_od_dict[e] for g in el_indices) \
                            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[(e, 1, t)] \
                                for od in edge_to_od_dict[e] for g in el_indices)
            
            obj[(e, t, 'R')] = sum( y[(od, g, e, 0, t)] * (1 - alpha[e, t]) * tau[e, t] \
                                for od in edge_to_od_dict[e] for g in el_indices) \
                                + sum( y[(od, g, e, 0, t)] * tau[e, t] \
                                    for od in edge_to_od_dict[e] for g in in_indices)

            obj[(e, t, 'I')] = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[(e, 0, t)] + tau[e, t]) \
                                for od in edge_to_od_dict[e] for g in in_indices) \
                            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[(e, 1, t)] \
                                for od in edge_to_od_dict[e] for g in in_indices)

            obj[(e, t)] = lambda_E * obj[(e, t, 'E')] - lambda_R * obj[(e, t, 'R')] + lambda_I * obj[(e, t, 'I')]  
        
        obj[(e, 'E')] = sum([obj[(e, t, 'E')] for t in range(T)])
        obj[(e, 'R')] = sum([obj[(e, t, 'R')] for t in range(T)])
        obj[(e, 'I')] = sum([obj[(e, t, 'I')] for t in range(T)])
        
        obj[e] = lambda_E * obj[(e, 'E')] - lambda_R * obj[(e, 'R')] + lambda_I * obj[(e, 'I')]  
    
    obj['total'] = sum([obj[e] for e in range(num_edges)])
    
    return obj


def compute_obj_DBCP_at_e_t(y, travel_times, VoT_array, tau, alpha, edge_to_od_dict, lambda_list, e, t):
    obj = {}
    
    lambda_E, lambda_R, lambda_I = lambda_list
    
    obj_E = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[0] + (1 - alpha) * tau) \
                                for od in edge_to_od_dict[e] for g in el_indices) \
                            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[1] \
                                for od in edge_to_od_dict[e] for g in el_indices)
    
    obj_R = sum( y[(od, g, e, 0, t)] * (1 - alpha) * tau \
                        for od in edge_to_od_dict[e] for g in el_indices) \
                        + sum( y[(od, g, e, 0, t)] * tau \
                            for od in edge_to_od_dict[e] for g in in_indices)

    obj_I = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[0] + tau) \
                        for od in edge_to_od_dict[e] for g in in_indices) \
                    + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[1] \
                        for od in edge_to_od_dict[e] for g in in_indices)

    obj = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I

    return obj, obj_E, obj_R, obj_I


In [50]:
def extract_y_at_e_t(y, e, t):
    
    y_at_e_t = {}
    
    for key in y.keys():
        if key[2] == e and key[4] == t:
            y_at_e_t[key] = y[key]
    
    return y_at_e_t

In [51]:
# Compute aggregate flows and other characteristics

y_agg_DBCP_init, x_DBCP_init \
    = compute_y_agg_x(y = y_DBCP_init, \
                      edge_to_od_dict = edge_to_od_dict,\
                      num_edges = num_edges, \
                      T = T)

travel_times_DBCP_init, avg_travel_times_DBCP_init \
    = compute_travel_times(y_agg = y_agg_DBCP_init, \
                           x = x_DBCP_init, \
                           edge_to_od_dict = edge_to_od_dict,\
                           coeff_input = coeff_input, \
                           num_gp_lanes = num_gp_lanes,\
                           num_edges = num_edges, \
                           T = T)

percent_on_express_DBCP_init \
    = compute_percent_on_express(y_agg = y_agg_DBCP_init,\
                                 x = x_DBCP_init, \
                                 num_edges = num_edges, \
                                 T = T)

lambda_list = [lambda_E, lambda_R, lambda_I]

# obj_DBCP_init_together \
#     = compute_obj_DBCP_together(y = y_DBCP_init, \
#                        x = x_DBCP_init, \
#                        travel_times = travel_times_DBCP_init, \
#                        VoT_array = VoT_array, \
#                        tau = argmin_tau, \
#                        alpha = init_alpha, \
#                        edge_to_od_dict = edge_to_od_dict, \
#                        lambda_list = lambda_list, \
#                        num_edges = num_edges, \
#                        T = T)

# obj_DBCP_init \
#     = compute_obj_DBCP(y = y_DBCP_init, \
#                        x = x_DBCP_init, \
#                        travel_times = travel_times_DBCP_init, \
#                        VoT_array = VoT_array, \
#                        tau = argmin_tau, \
#                        alpha = init_alpha, \
#                        edge_to_od_dict = edge_to_od_dict, \
#                        lambda_list = lambda_list, \
#                        num_edges = num_edges, \
#                        T = T)

# print("travel_times_DBCP_init:", travel_times_DBCP_init)

obj_DBCP_init = {}
for e in range(num_edges):
    for t in range(T):
        
        travel_times_DBCP_init_at_e_t = [travel_times_DBCP_init[(e, k, t)] for k in [0, 1]]

        obj_DBCP_init[(e, t)], obj_DBCP_init[(e, t, 'E')], \
                obj_DBCP_init[(e, t, 'R')], obj_DBCP_init[(e, t, 'I')] \
            = compute_obj_DBCP_at_e_t(y = y_DBCP_init, \
                                      travel_times = travel_times_DBCP_init_at_e_t, \
                                      VoT_array = VoT_array, \
                                      tau = argmin_tau[e, t], \
                                      alpha = init_alpha[e, t], \
                                      edge_to_od_dict = edge_to_od_dict, \
                                      lambda_list = lambda_list, \
                                      e = e, \
                                      t = t)

    obj_DBCP_init[(e, 'E')] = sum([obj_DBCP_init[(e, t, 'E')] for t in range(T)])
    obj_DBCP_init[(e, 'R')] = sum([obj_DBCP_init[(e, t, 'R')] for t in range(T)])
    obj_DBCP_init[(e, 'I')] = sum([obj_DBCP_init[(e, t, 'I')] for t in range(T)])

    obj_DBCP_init[e] = lambda_E * obj_DBCP_init[(e, 'E')] - lambda_R * obj_DBCP_init[(e, 'R')] + lambda_I * obj_DBCP_init[(e, 'I')]  

obj_DBCP_init['total'] = sum([obj_DBCP_init[e] for e in range(num_edges)])


# print("percent_on_express_DBCP_init:\n", percent_on_express_DBCP_init)

# obj_DBCP_init['total']

In [52]:
# print(obj_DBCP_init == obj___from_func)

print("obj_CBCP_opt[total]:", obj_CBCP_opt['total'])
print("obj_DBCP_init[total]:", obj_DBCP_init['total'])
# obj_DBCP_init['total']

obj_CBCP_opt[total]: 772369.2906431704
obj_DBCP_init[total]: 772334.0932094305


In [53]:
# for k in np.linspace(0, 10, 5):
#     print(k)

In [54]:
# y_DBCP_init_at_e_t, VoT_in_boundary[e, t], VoT_el_boundary[e, t], init_alpha[e, t] \
#             = y_agg_to_y_DBCP_at_e_t(y_agg_in_at_e_t = y_agg_CBCP_opt[(e, 0, t, 'in')], \
#                                      y_agg_el_at_e_t = y_agg_CBCP_opt[(e, 0, t, 'el')], \
#                                      VoTs_demands_annotated_in = dict_VoTs_demands_annotated[(e, t, "in")], \
#                                      VoTs_demands_annotated_el = dict_VoTs_demands_annotated[(e, t, "el")], \
#                                      e = e, \
#                                      t = t)

In [55]:
y_DBCP_init_min = {}

obj_DBCP_init_min = np.zeros((num_edges, T))

num_y_toggle = 101

for e in range(num_edges):
    for t in range(T):
        
        print()
        print("e:", e)
        print("t:", t)
        print()
        
        # Compute obj (DBCP) for DBCP init, as baseline:
        
        travel_times_DBCP_init_at_e_t = [travel_times_DBCP_init[(e, k, t)] for k in [0, 1]]
        
        y_DBCP_init_min_at_e_t = extract_y_at_e_t(y_DBCP_init, e, t)
        
        obj_DBCP_init_min_at_e_t, _, _, _ \
            = compute_obj_DBCP_at_e_t(y = y_DBCP_init, \
                                      travel_times = travel_times_DBCP_init_at_e_t, \
                                      VoT_array = VoT_array, \
                                      tau = argmin_tau[e, t], \
                                      alpha = init_alpha[e, t], \
                                      edge_to_od_dict = edge_to_od_dict, \
                                      lambda_list = lambda_list, \
                                      e = e, \
                                      t = t)
        
        if y_agg_boundary_to_toggle[(e, t)]:
            y_agg_el_ex_arr = np.linspace(y_agg_el_ex_low[e, t], y_agg_el_ex_high[e, t], num_y_toggle)
            
            for y_agg_el_ex in y_agg_el_ex_arr:
                
                y_agg_in_ex = x_DBCP_init[(e, 0, t)] - y_agg_el_ex
                
#                 print("y_agg_el_ex:", y_agg_el_ex)
#                 print("y_agg_in_ex:", y_agg_in_ex)
                
                y_DBCP_toggle_at_e_t, _, _, _ \
                    = y_agg_to_y_DBCP_at_e_t(y_agg_in_at_e_t = y_agg_in_ex, \
                                             y_agg_el_at_e_t = y_agg_el_ex, \
                                             VoTs_demands_annotated_in = dict_VoTs_demands_annotated[(e, t, "in")], \
                                             VoTs_demands_annotated_el = dict_VoTs_demands_annotated[(e, t, "el")], \
                                             e = e, \
                                             t = t)

                obj_toggle_at_e_t, _, _, _ \
                    = compute_obj_DBCP_at_e_t(y = y_DBCP_toggle_at_e_t, \
                                              travel_times = travel_times_DBCP_init_at_e_t, \
                                              VoT_array = VoT_array, \
                                              tau = argmin_tau[e, t], \
                                              alpha = init_alpha[e, t], \
                                              edge_to_od_dict = edge_to_od_dict, \
                                              lambda_list = lambda_list, \
                                              e = e, \
                                              t = t)
                
                if obj_toggle_at_e_t < obj_DBCP_init_min_at_e_t:
                    
                    y_DBCP_init_min_at_e_t = y_DBCP_toggle_at_e_t
                    obj_DBCP_init_min_at_e_t = obj_toggle_at_e_t
        
        y_DBCP_init_min = {**y_DBCP_init_min, **y_DBCP_init_min_at_e_t}
        obj_DBCP_init_min[e, t] = obj_DBCP_init_min_at_e_t




e: 0
t: 0


e: 0
t: 1


e: 0
t: 2


e: 0
t: 3


e: 0
t: 4


e: 1
t: 0


e: 1
t: 1


e: 1
t: 2


e: 1
t: 3


e: 1
t: 4


e: 2
t: 0


e: 2
t: 1


e: 2
t: 2


e: 2
t: 3


e: 2
t: 4


e: 3
t: 0


e: 3
t: 1


e: 3
t: 2


e: 3
t: 3


e: 3
t: 4


e: 4
t: 0


e: 4
t: 1


e: 4
t: 2


e: 4
t: 3


e: 4
t: 4


e: 5
t: 0


e: 5
t: 1


e: 5
t: 2


e: 5
t: 3


e: 5
t: 4


e: 6
t: 0


e: 6
t: 1


e: 6
t: 2


e: 6
t: 3


e: 6
t: 4



In [56]:
# y_DBCP_init_min

print("obj_CBCP_opt[total]:\n", obj_CBCP_opt['total'])
print()
print("obj_DBCP_init[total]:\n", obj_DBCP_init['total'])
print()
print("np.sum(obj_DBCP_init_min):\n", np.sum(obj_DBCP_init_min))

# np.sum(obj_DBCP_init_min)
obj_CBCP_opt['total'] - np.sum(obj_DBCP_init_min)

obj_CBCP_opt[total]:
 772369.2906431704

obj_DBCP_init[total]:
 772334.0932094305

np.sum(obj_DBCP_init_min):
 771667.2019657913


702.0886773790698

# DBCP Optimization

In [114]:
def proj_tau_alpha(T, num_edges, tau, alpha, tau_max):

#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert tau.shape[0] == num_edges, "tau must have length equal to the number of edges."
    assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
    tau_feas = cp.Variable((num_edges, T))
    
    func = cp.sum_squares(tau_feas - tau)

    objective = cp.Minimize(func)

    constraints = []
    constraints += [tau_feas >= 0.0]
    constraints += [tau_feas <= tau_max]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()
    
    alpha_feas = np.clip(alpha, 0.0, 1.0)

    print()
    print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=3))
    print()
    print("alpha_feas:", alpha_feas)
    print()
    
#     print("alpha:", alpha)
#     print()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=3), \
            np.round(np.maximum(alpha_feas, 0.0), decimals=3)
    
#     return tau_feas.value, alpha_feas



def proj_tau_alpha_at_e_t(T, num_edges, tau, alpha, tau_max):

#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

#     assert tau == num_edges, "tau must have length equal to the number of edges."
#     assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
    tau_feas = cp.Variable(1)
    
    func = cp.sum_squares(tau_feas - tau)

    objective = cp.Minimize(func)

    constraints = []
    constraints += [tau_feas >= 0.0]
    constraints += [tau_feas <= tau_max]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()
    
    alpha_feas = np.clip(alpha, 0.0, 1.0)

    print()
    print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=3))
    print()
    print("alpha_feas:", alpha_feas)
    print()
    
    print("np.round(np.maximum(tau_feas.value, 0.0), decimals=3):", \
          np.round(np.maximum(tau_feas.value, 0.0), decimals=3))
    print("np.round(np.maximum(alpha_feas, 0.0), decimals=3):", \
         np.round(np.maximum(alpha_feas, 0.0), decimals=3))
    
#     print("alpha:", alpha)
#     print()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=3), \
            np.round(np.maximum(alpha_feas, 0.0), decimals=3)
    
#     return tau_feas.value, alpha_feas



In [115]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0

def solve_DBCP_at_e_t(num_gp_lanes, tau, alpha, od_to_edges_list_full, \
                      demand_array, VoT_array, el_indices, in_indices, \
                      coeff_input, e, t):
    
    assert tau.shape == alpha.shape, "We must have tau.shape == alpha.shape!"
    assert np.all((alpha >= -1E-3) & (alpha <= 1.001))
    
    alpha = np.clip(alpha, 0.0, 1.0)
    
#     print("tau (in solve_DBCP_direct):", tau)
    
#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)

    coeff = coeff_input
    
    print("coeff:\n", coeff)
    print()
    
#     assert VoT_array.shape[2] == T, "We should have VoT_array.shape[2] == T."
    assert demand_array.shape == VoT_array[:, :, 0].shape, "demand_array and VoT_array[:, :, 0] should have identical shape."
    assert np.all(demand_array > 0.0), "Each entry of demand_array must be strictly positive."
    assert np.all(tau >= 0.0), "Each entry of tau must be non-negative."
#     assert num_el <= demand_array.shape[1], "num_el, the number of eligible income groups, should not exceed \
#                                             demand_array.shape[1], which should equal the number of income groups."
    
    num_groups = demand_array.shape[1]
    
    ## Variable indices:
    # y indices: (od, g, e, k, t) = (od, income group, edge, "lane", time)
    
#     od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
#                           for od in range(od_to_edges_array.shape[0])]
    
#     edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
#                    for e in range(num_edges)]
    
    num_od = len(od_to_edges_list_full)
    
    # Variables:
    y = {}
    for od in range(num_od):
        for g in el_indices:
            for k in [0, 1, 2]:
                y[(od, g, e, k, t)] = cp.Variable(1)
        for g in in_indices:
            for k in [0, 1]:
                y[(od, g, e, k, t)] = cp.Variable(1)
    
    x = {}
    for k in [0, 1]:
        x[(e, k, t)] = cp.Variable(1)

    # Objective:
    func = 0.0
    func += coeff[0, e] * x[(e, 0, t)] \
            + 0.5 * coeff[1, e] * cp.square(cp.maximum(x[(e, 0, t)] - coeff[2, e], 0))
    func += coeff[0, e] * x[(e, 1, t)] \
            + 0.5 * coeff[1, e] * num_gp_lanes * cp.square(cp.maximum(x[(e, 1, t)]/num_gp_lanes - coeff[2, e], 0))
    
#     print("VoT_array.shape:", VoT_array.shape)
#     print("el_indices:", el_indices)
#     print("in_indices:", in_indices)

    
    for od in range(num_od):
        for g in el_indices:
            
#             print("alpha[e, t]:", alpha[e, t])
#             print("tau[e, t]:", tau[e, t])
#             print("VoT_array[od, g, t]:", VoT_array[od, g, t])
            
            func += (1 - alpha[e, t]) * tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g, t]
#             func += tau[e, t] * y[(od, g, e, 1, t)] / VoT_array[od, g, t]
#         for g in in_indices:
#             func += tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g, t]

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) for g in el_indices for k in [0, 1, 2]]
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) for g in in_indices for k in [0, 1]]
    constraints += [x[(e, k, t)] >= 0.0 for k in [0, 1]]
    
#     print()
#     print("demand_array:", demand_array)
#     print()
#     print("alpha:", alpha)
#     print()
#     print("el_indices:", el_indices)
#     print()
#     print("in_indices:", in_indices)
#     print()
#     print("edge_to_ods:", edge_to_ods)
#     print()
#     print("od_to_edges_list_full:", od_to_edges_list_full)
#     print()
    
    od_g_e_t_list = []
    
    ## Edge contraints:
    constraints += [sum( y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in el_indices) \
                        + sum( y[(od, g, e, 0, t)] for od in edge_to_ods[e] for g in in_indices ) \
                        == x[(e, 0, t)] ]
    constraints += [sum( y[(od, g, e, 2, t)] for od in edge_to_ods[e] for g in el_indices) \
                        + sum( y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in in_indices ) \
                        == x[(e, 1, t)] ]

    ## Group flow constraints:
    for od in edge_to_ods[e]:
        for g in el_indices:
#                     print("(od, g, e, t):", od, g, e, t)
            assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
            od_g_e_t_list += [(od, g, e, t)]

#                     if (od, g, e, t) == (0, 0, 0, 0):
#                     if (e, t) == (2, 0) or (2, 1) or (2, 3):
#                         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g]]

#                 for g in in_indices:
# #                     print("(od, g, e, t):", od, g, e, t)
#                     assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
#                     od_g_e_t_list += [(od, g, e, t)]

#                     constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g]]

    constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g] \
                    for od in edge_to_ods[e] for g in el_indices]
    constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
                    for od in edge_to_ods[e] for g in in_indices]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
#     result = prob.solve(solver = cp.MOSEK, verbose = True)
    result = prob.solve(solver = cp.MOSEK)
    
#     for variable in prob.variables():
#         print("Variable %s" % variable.name())
    
    assert prob.status != "infeasible", "problem.status should not be infeasible."
    assert prob.status != "unbounded", "problem.status should not be unbounded."
    print()
    print("prob.status:", prob.status)

    # Extract Values:
    y_values = {}
    for od in edge_to_ods[e]:
        for g in el_indices:
            for k in [0, 1, 2]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)
        for g in in_indices:
            for k in [0, 1]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)

    return y_values


In [116]:
def welfare_obj_at_e_t(num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, alpha, \
                       demand_array, VoT_array, num_el, od_to_edges_array, y, \
                       coeff_input, e, t):

#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)

    assert np.all((alpha >= -1E-3) & (alpha <= 1.001)), "We must have alpha in [0, 1] component-wise!"
    
    coeff = coeff_input
    
    assert len(od_to_edges_array.shape) == 2, "od_to_edges should be 2-dimensional."
    assert od_to_edges_array.shape[1] == 2, "od_to_edges second dimension should be for start and end edges."
    
    edge_to_od_dict = {}
    edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                           if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

#     assert len(tau.shape) == 2, "tau should be 2-dimensional."
#     assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
#     assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros(2)
    x[0] += sum((y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)]) for od in edge_to_od_dict[e] for g in el_indices)
    x[0] += sum(y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in in_indices)

    x[1] += sum(y[(od, g, e, 2, t)] for od in edge_to_od_dict[e] for g in el_indices)
    x[1] += sum(y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] for g in in_indices)
        
#     print()
#     print("in_indices:", in_indices)
#     print("el_indices:", el_indices)
#     print()
    
    ## Compute lane latencies:
    
    # TODO: To edit below:
    # Update: The culprint is ell[0], which is a 10 x 1 array instead of being scalar.
    
    ell = np.zeros(2)
    ell[0] = coeff[0, e] + coeff[1, e] * max(x[0] - coeff[2, e], 0)
    ell[1] = coeff[0, e] + coeff[1, e] * max(x[1]/num_gp_lanes - coeff[2, e], 0)    

    # Computing obj_E, obj_I, obj_R, and welfare at edge e and time t:
    
    obj_E = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * ell[0] + (1 - alpha) * tau ) \
                for od in edge_to_od_dict[e] for g in el_indices) \
            + sum( y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * ell[0] + tau) \
                for od in edge_to_od_dict[e] for g in el_indices) \
            + sum( y[(od, g, e, 2, t)] * VoT_array[od, g, t] * ell[1] \
                  for od in edge_to_od_dict[e] for g in el_indices)
    obj_I = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * ell[0] + tau) \
                for od in edge_to_od_dict[e] for g in in_indices) \
            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * ell[1] \
                for od in edge_to_od_dict[e] for g in in_indices)
    obj_R = sum( y[(od, g, e, 0, t)] * (1 - alpha) * tau \
                for od in edge_to_od_dict[e] for g in el_indices) \
            + sum( y[(od, g, e, 0, t)] * tau \
                for od in edge_to_od_dict[e] for g in in_indices)

    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I

#     print()
#     print("In the welfare_obj_at_e_t function, -1:")
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return welfare, obj_E, obj_R, obj_I


##  <font color='red'> (August 3) Run zeroth-order gradient descent at each e and t. Start by testing e=1 and t=1.</font> 

## <font color='red'> To edit below.</font>

In [117]:
T = 5

# num_iters = 1000
# num_iters = 100
num_iters = 200
# num_iters = 3

num_el = 2
num_groups = demand_array.shape[1]

assert VoT_array.shape[2] == T, "We should have VoT_array.shape[2] == T"
assert demand_array.shape == VoT_array[:, :, 0].shape, "demand_array and VoT_array[:, :, 0] should have the same shape."

num_ods = demand_array.shape[0]
group_indices = list(range(demand_array.shape[1]))
# num_edges = 6
num_edges = 7
num_gp_lanes = 3

# tau = np.array([0.2, 0.6, 0.5, 0.8, 0.4, 0.5])
# assert tau.shape[0] == num_edges, "tau must have a num_edges number of entries."

error_bound = 1E-2
diffs_num_cols = 5

demand_edges_array = np.zeros(num_edges)

# demand_array_temp = np.ones(demand_array.shape)
# demand_array_temp = np.random.uniform(low=0.05, high=0.5, size=demand_array.shape)
demand_array_temp = demand_array


## coeff_input: const, slope, x-coordinate of transition point

# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))


coeff_input = np.zeros((3, num_edges))
for counter, city in enumerate(dict_latency_params.keys()):
    coeff_input[0, counter] = dict_latency_params[city]["Latency (at bend)"]
    coeff_input[1, counter] = dict_latency_params[city]["Slope (after bend)"]
    coeff_input[2, counter] = dict_latency_params[city]["Flow (at bend)"]


# def solve_DBCP_direct(T, num_edges, num_gp_lanes, tau, alpha, od_to_edges_array, \
#                       demand_array, VoT_array, num_el, coeff_input):
# Already defined: T, num_edges, num_gp_lanes, tau, alpha, od_to_edges_array, \
#                       demand_array, VoT_array

    
# print("od_to_edges_array[1, 0]:", int(od_to_edges_array[1, 0]))
# print("range(od_to_edges_array[1, 0], od_to_edges_array[1, 1] + 1):",\
#      range(int(od_to_edges_array[1, 0]), int(od_to_edges_array[1, 1]) + 1))

od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
                         for od in range(num_ods)]
edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
               for e in range(num_edges)]
for e in range(num_edges):
    demand_edges_array[e] = sum([np.sum(demand_array_temp[od, :]) for od in range(num_ods) \
                                 if od in edge_to_ods[e]])

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

tau_max_from_latency = np.zeros((num_edges, T))
VoT_max_el = np.zeros((num_edges, T))
for e in range(num_edges):
    for t in range(T):
        VoT_max_el[e, t] = max([VoT_array[od_index, group_index, t] for od_index in edge_to_ods[e] \
                                for group_index in el_indices])
        tau_max_from_latency[e, t] = VoT_max_el[e, t] * (latency_max(demand_edges_array[e], coeff_input[:, e]) - coeff_input[0, e]) \

tau_max_monetary_value = 15.0
# fraction_tau_max = tau_max_monetary_value / np.max(tau_max_from_latency)

tau_upper_limit = np.minimum(tau_max_from_latency, tau_max_monetary_value)

print("tau_upper_limit:", tau_upper_limit)

d = 2

tau = np.zeros((num_edges, T, num_iters))
tau_perturbed = np.zeros((num_edges, T, num_iters))

alpha = np.zeros((num_edges, T, num_iters))
alpha_perturbed = np.zeros((num_edges, T, num_iters))

fraction_tau_max = 0.5

# tau[:, :, 0] = np.random.triangular(np.zeros((num_edges, T)), tau_upper_limit * fraction_tau_max, tau_upper_limit)
# print()
# print("tau[:, :, 0]:", tau[:, :, 0])
# print()
# print("np.sum(tau[:, :, 0]):", np.sum(tau[:, :, 0]))

# od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1 )) \
#                          for od in range(od_to_edges_array.shape[0])]


# # To disable when restarting from scratch
# tau[:, :, 0] = tau_next_init
# alpha[:, :, 0] = alpha_next_init
# eta[0] = eta_bar * (index_next_init+1)**(-1/2) * d**(-1)
# delta[0] = delta_bar * (index_next_init+1)**(-1/4) * d**(-1/2)

tau_upper_limit: [[0.31942364 0.31942364 0.31942364 0.31942364 0.31942364]
 [0.24983105 0.24983105 0.24983105 0.24983105 0.24983105]
 [0.99953381 0.99953381 0.99953381 0.99953381 0.99953381]
 [1.02805916 1.02805916 1.02805916 1.02805916 1.02805916]
 [1.6042613  1.6042613  1.6042613  1.6042613  1.6042613 ]
 [0.19224693 0.19224693 0.19224693 0.19224693 0.19224693]
 [0.21782897 0.21782897 0.21782897 0.21782897 0.21782897]]


In [118]:
print("argmin_tau:\n", argmin_tau)
print()
print("init_alpha:\n", init_alpha)

argmin_tau:
 [[0.1  0.19 0.29 0.1  0.04]
 [0.04 0.07 0.21 0.06 0.15]
 [0.79 0.66 0.98 0.07 0.79]
 [0.54 0.8  0.43 0.24 0.3 ]
 [0.74 0.46 1.2  1.   0.39]
 [0.04 0.13 0.1  0.07 0.08]
 [0.15 0.14 0.11 0.1  0.14]]

init_alpha:
 [[0.98195575 0.98230519 0.98244702 0.98271836 0.94843263]
 [0.98195575 0.98243166 0.98244702 0.94466175 0.94654435]
 [0.94552141 0.98230519 0.98244702 0.94397746 0.94358402]
 [0.94250443 0.98379752 0.98392739 0.94084577 0.94358402]
 [0.94407335 0.98230519 0.98392739 0.94625608 0.9440454 ]
 [0.9438093  0.98230519 0.98353697 0.94895338 0.94340947]
 [0.94729265 0.98283659 0.98284063 0.98250467 0.94691761]]


In [148]:
time_1 = time.time()

welfares = np.zeros((num_edges, T, num_iters))

## Set lambdas:
# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

## Check lambda:
print("lambda_E:", lambda_E)
print("lambda_R:", lambda_R)
print("lambda_I:", lambda_I)
print()

## Initialize tau, alpha values:

# filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))
# # filename_segment = '4654283787'

# directory_inits = '../data/opt_tolls_subsidies_metrics/'
# df_inits = pd.read_csv(directory_inits + 'inits___' + filename_segment + '.csv')

# print("filename_segment:", filename_segment)


tau[:, :, 0] = argmin_tau
alpha[:, :, 0] = init_alpha

print()
print("tau[:, :, 0]:", tau[:, :, 0])
print("alpha[:, :, 0]:", alpha[:, :, 0])


delta = np.zeros(num_iters)
eta = np.zeros(num_iters)

# eta_bar = 1E-5
# delta_bar = 1E-5

eta_bar = 0.1
delta_bar = 0.1


for e in range(num_edges):
    for t in range(T):

# for e in [0]:
#     for t in [0]:
        for i in range(num_iters-1):
        
            time_1_at_e_t = time.time()

            print()
            print("Iter:", i)

            eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
            delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
            w_i_unnormalized = np.random.randn(2)
            w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
            
        #     print("w_i:", w_i)
            tau_perturbed[e, t, i] = tau[e, t, i] + delta[i] * w_i[0]
            alpha_perturbed[e, t, i] = alpha[e, t, i] + delta[i] * w_i[1]

        #     if tau_perturbed[e, t, i] < alpha_perturbed[e, t, i] or tau_perturbed[e, t, i] < 0 alpha_perturbed[e, t, i] < 0:

            print("i:\n", i)
            print("tau[e, t, i], before projection:\n", tau[e, t, i])
            print("alpha[e, t, i], before projection:\n", alpha[e, t, i])
            print("tau_perturbed[e, t, i], before projection:\n", tau_perturbed[e, t, i])
            print("alpha_perturbed[e, t, i], before projection:\n", alpha_perturbed[e, t, i])
            
            # TODO: To edit below:
            
            print("tau_perturbed[e, t, i]:", tau_perturbed[e, t, i])
            print("alpha_perturbed[e, t, i]:", alpha_perturbed[e, t, i])
            print("tau_upper_limit:", tau_upper_limit)

            tau_perturbed[e, t, i], alpha_perturbed[e, t, i] \
                = proj_tau_alpha_at_e_t(T, num_edges, tau_perturbed[e, t, i], alpha_perturbed[e, t, i], \
                                        tau_max = tau_upper_limit[e, t])

            print("tau[e, t, i], after projection:\n", tau[e, t, i])
            print("alpha[e, t, i], after projection:\n", alpha[e, t, i])
            print("tau_perturbed[e, t, i], after projection:\n", tau_perturbed[e, t, i])
            print("alpha_perturbed[e, t, i], after projection:\n", alpha_perturbed[e, t, i])
            
#             def solve_DBCP_at_e_t(num_gp_lanes, tau, alpha, od_to_edges_list_full, \
#                       demand_array, VoT_array, el_indices, in_indices, \
#                       coeff_input, e, t):

            print()
#             print("demand_array_temp:", demand_array_temp)
            print("VoT_array.shape:", VoT_array.shape)
            print()

            y_values_at_e_t = \
                solve_DBCP_at_e_t(num_gp_lanes, tau[:, :, i], alpha[:, :, i], od_to_edges_list_full, \
                                         demand_array_temp, VoT_array, el_indices, in_indices, \
                                         coeff_input, e, t)

            y_perturbed_values_at_e_t = \
                solve_DBCP_at_e_t(num_gp_lanes, tau_perturbed[:, :, i], alpha_perturbed[:, :, i], od_to_edges_list_full, \
                                                   demand_array_temp, VoT_array, el_indices, in_indices, \
                                                   coeff_input, e, t)
            
            
            
#             def welfare_obj_at_e_t(num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, alpha, \
#                        demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                        coeff_input, e, t):

            welfare_at_e_t, obj_E_at_e_t, obj_R_at_e_t, obj_I_at_e_t = \
                welfare_obj_at_e_t(num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                                                       tau[e, t, i], alpha[e, t, i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                                                       y = y_values_at_e_t, coeff_input = coeff_input, e = e, t = t)
            
            welfare_perturbed_at_e_t, obj_E_perturbed_at_e_t, obj_R_perturbed_at_e_t, obj_I_perturbed_at_e_t \
                = welfare_obj_at_e_t(num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                              tau_perturbed[e, t, i], alpha_perturbed[e, t, i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                              y = y_perturbed_values_at_e_t, coeff_input = coeff_input, e = e, t = t)
            
            print()
            print("welfare_at_e_t:", welfare_at_e_t)
            print("obj_E_at_e_t:", obj_E_at_e_t)
            print("obj_R_at_e_t:", obj_R_at_e_t)
            print("obj_I_at_e_t:", obj_I_at_e_t)
            print()
            print("welfare_perturbed_at_e_t:", welfare_perturbed_at_e_t)
            print("obj_E_perturbed_at_e_t:", obj_E_perturbed_at_e_t)
            print("obj_R_perturbed_at_e_t:", obj_R_perturbed_at_e_t)
            print("obj_I_perturbed_at_e_t:", obj_I_perturbed_at_e_t)
            print()

            welfares[e, t, i] = welfare_at_e_t
            
#             print("quantity:", tau[e, t, i] - eta[i] * (d/delta[i]) * w_i[0] \
#                                 * (welfare_perturbed_at_e_t - welfare_at_e_t))

            tau[e, t, i+1] = tau[e, t, i] - eta[i] * (d/delta[i]) * w_i[0] \
                                * (welfare_perturbed_at_e_t - welfare_at_e_t)

            alpha[e, t, i+1] = alpha[e, t, i] - eta[i] * (d/delta[i]) * w_i[1] \
                                * (welfare_perturbed_at_e_t - welfare_at_e_t)

            print("tau[:, :, i+1], before projection:\n", tau[:, :, i+1])
            print("alpha[:, :, i+1], before projection:\n", alpha[:, :, i+1])
            
            # TODO: To edit below:

            tau[:, :, i+1], alpha[:, :, i+1] \
                = proj_tau_alpha_at_e_t(T, num_edges, tau[:, :, i+1], alpha[:, :, i+1], \
                                        tau_max = tau_upper_limit)

            print("tau[:, :, i+1], after projection:\n", tau[:, :, i+1])
            print("alpha[:, :, i+1], after projection:\n", alpha[:, :, i+1])

            if i >= diffs_num_cols + 2:
                tau_diffs = np.max(np.absolute(tau[:, :, i - diffs_num_cols : i-1] - tau[:, :, i - diffs_num_cols+1 : i]), axis = 2)
                alpha_diffs = alpha[:, :, i-diffs_num_cols : i-1] - alpha[:, :, i-diffs_num_cols+1 : i]

        #         print("tau[:, :, 0:10]:", tau[:, :, 0:10])
        #         print("alpha[:, :, 0:10]:", alpha[:, :, 0:10])
                print("tau_diffs:\n", tau_diffs)
                print("alpha_diffs:\n", alpha_diffs)
                print()

                if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(alpha_diffs))) < error_bound:
                    print("Within error bound.")
                    break

            time_2_at_e_t = time.time()

            print()
            print("Iteration count:", i)
            print("Time:", time_2_at_e_t - time_1_at_e_t)
            print()

time_2 = time.time()
print("Time:", time_2 - time_1)


lambda_E: 1.0
lambda_R: 1.0
lambda_I: 1.0


tau[:, :, 0]: [[0.1  0.19 0.29 0.1  0.04]
 [0.04 0.07 0.21 0.06 0.15]
 [0.79 0.66 0.98 0.07 0.79]
 [0.54 0.8  0.43 0.24 0.3 ]
 [0.74 0.46 1.2  1.   0.39]
 [0.04 0.13 0.1  0.07 0.08]
 [0.15 0.14 0.11 0.1  0.14]]
alpha[:, :, 0]: [[0.98195575 0.98230519 0.98244702 0.98271836 0.94843263]
 [0.98195575 0.98243166 0.98244702 0.94466175 0.94654435]
 [0.94552141 0.98230519 0.98244702 0.94397746 0.94358402]
 [0.94250443 0.98379752 0.98392739 0.94084577 0.94358402]
 [0.94407335 0.98230519 0.98392739 0.94625608 0.9440454 ]
 [0.9438093  0.98230519 0.98353697 0.94895338 0.94340947]
 [0.94729265 0.98283659 0.98284063 0.98250467 0.94691761]]

Iter: 0
i:
 0
tau[e, t, i], before projection:
 0.1
alpha[e, t, i], before projection:
 0.9819557531646372
tau_perturbed[e, t, i], before projection:
 0.16817713583377566
alpha_perturbed[e, t, i], before projection:
 0.9631973377330795
tau_perturbed[e, t, i]: 0.16817713583377566
alpha_perturbed[e, t, i]: 0.9631973377330


prob.status: optimal

welfare_at_e_t: 8196.870895119311
obj_E_at_e_t: 55.10023196109217
obj_R_at_e_t: 184.2098121357942
obj_I_at_e_t: 8325.980475294013

welfare_perturbed_at_e_t: 8196.870895119311
obj_E_perturbed_at_e_t: 55.10023196109217
obj_R_perturbed_at_e_t: 184.2098121357942
obj_I_perturbed_at_e_t: 8325.980475294013

tau[:, :, i+1], before projection:
 [[0.152 0.1   0.1   0.1   0.1  ]
 [0.1   0.1   0.1   0.1   0.1  ]
 [0.1   0.1   0.1   0.1   0.1  ]
 [0.1   0.1   0.1   0.1   0.1  ]
 [0.1   0.1   0.1   0.1   0.1  ]
 [0.1   0.1   0.1   0.1   0.1  ]
 [0.1   0.1   0.1   0.1   0.1  ]]
alpha[:, :, i+1], before projection:
 [[0.982 0.982 0.982 0.983 0.948]
 [0.982 0.982 0.982 0.945 0.947]
 [0.946 0.982 0.982 0.944 0.944]
 [0.943 0.984 0.984 0.941 0.944]
 [0.944 0.982 0.984 0.946 0.944]
 [0.944 0.982 0.984 0.949 0.943]
 [0.947 0.983 0.983 0.983 0.947]]

tau_feas.value: [0.101]

alpha_feas: [[0.982 0.982 0.982 0.983 0.948]
 [0.982 0.982 0.982 0.945 0.947]
 [0.946 0.982 0.982 0.944 0.944]



prob.status: optimal

welfare_at_e_t: 8196.870895119311
obj_E_at_e_t: 55.10023196109217
obj_R_at_e_t: 33.93338644606735
obj_I_at_e_t: 8175.7040496042855

welfare_perturbed_at_e_t: 8196.87089511931
obj_E_perturbed_at_e_t: 55.10023196109217
obj_R_perturbed_at_e_t: 32.721479787279236
obj_I_perturbed_at_e_t: 8174.492142945497

tau[:, :, i+1], before projection:
 [[0.028 0.01  0.01  0.01  0.01 ]
 [0.01  0.01  0.01  0.01  0.01 ]
 [0.01  0.01  0.01  0.01  0.01 ]
 [0.01  0.01  0.01  0.01  0.01 ]
 [0.01  0.01  0.01  0.01  0.01 ]
 [0.01  0.01  0.01  0.01  0.01 ]
 [0.01  0.01  0.01  0.01  0.01 ]]
alpha[:, :, i+1], before projection:
 [[0.982 0.982 0.982 0.983 0.948]
 [0.982 0.982 0.982 0.945 0.947]
 [0.946 0.982 0.982 0.944 0.944]
 [0.943 0.984 0.984 0.941 0.944]
 [0.944 0.982 0.984 0.946 0.944]
 [0.944 0.982 0.984 0.949 0.943]
 [0.947 0.983 0.983 0.983 0.947]]

tau_feas.value: [0.011]

alpha_feas: [[0.982 0.982 0.982 0.983 0.948]
 [0.982 0.982 0.982 0.945 0.947]
 [0.946 0.982 0.982 0.944 0.944]


prob.status: optimal

welfare_at_e_t: 8196.87089511931
obj_E_at_e_t: 55.10023196109217
obj_R_at_e_t: 0.0
obj_I_at_e_t: 8141.770663158218

welfare_perturbed_at_e_t: 8196.87089511931
obj_E_perturbed_at_e_t: 55.10023196109217
obj_R_perturbed_at_e_t: 31.509573128491112
obj_I_perturbed_at_e_t: 8173.280236286709

tau[:, :, i+1], before projection:
 [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
alpha[:, :, i+1], before projection:
 [[0.982 0.982 0.982 0.983 0.948]
 [0.982 0.982 0.982 0.945 0.947]
 [0.946 0.982 0.982 0.944 0.944]
 [0.943 0.984 0.984 0.941 0.944]
 [0.944 0.982 0.984 0.946 0.944]
 [0.944 0.982 0.984 0.949 0.943]
 [0.947 0.983 0.983 0.983 0.947]]

tau_feas.value: [0.]

alpha_feas: [[0.982 0.982 0.982 0.983 0.948]
 [0.982 0.982 0.982 0.945 0.947]
 [0.946 0.982 0.982 0.944 0.944]
 [0.943 0.984 0.984 0.941 0.944]
 [0.944 0.982 0.984 0.946 0.944]
 [0.944 0.982 0.984 0.949 0.943]
 [0.947 0.983 0.983 0.98


prob.status: optimal

welfare_at_e_t: 8196.87089511931
obj_E_at_e_t: 55.10023196109217
obj_R_at_e_t: 0.0
obj_I_at_e_t: 8141.770663158218

welfare_perturbed_at_e_t: 8196.87089511931
obj_E_perturbed_at_e_t: 55.10023196109217
obj_R_perturbed_at_e_t: 30.297666469702996
obj_I_perturbed_at_e_t: 8172.06832962792

tau[:, :, i+1], before projection:
 [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
alpha[:, :, i+1], before projection:
 [[0.982 0.982 0.982 0.983 0.948]
 [0.982 0.982 0.982 0.945 0.947]
 [0.946 0.982 0.982 0.944 0.944]
 [0.943 0.984 0.984 0.941 0.944]
 [0.944 0.982 0.984 0.946 0.944]
 [0.944 0.982 0.984 0.949 0.943]
 [0.947 0.983 0.983 0.983 0.947]]

tau_feas.value: [0.]

alpha_feas: [[0.982 0.982 0.982 0.983 0.948]
 [0.982 0.982 0.982 0.945 0.947]
 [0.946 0.982 0.982 0.944 0.944]
 [0.943 0.984 0.984 0.941 0.944]
 [0.944 0.982 0.984 0.946 0.944]
 [0.944 0.982 0.984 0.949 0.943]
 [0.947 0.983 0.983 0.983

...


prob.status: optimal

welfare_at_e_t: 18139.97510687325
obj_E_at_e_t: 145.96507173312375
obj_R_at_e_t: 149.4011869256148
obj_I_at_e_t: 18143.41122206574

welfare_perturbed_at_e_t: 18139.97510687325
obj_E_perturbed_at_e_t: 145.96507173312375
obj_R_perturbed_at_e_t: 232.99470818161353
obj_I_perturbed_at_e_t: 18227.00474332174

tau[:, :, i+1], before projection:
 [[0.061 0.061 0.061 0.061 0.061]
 [0.061 0.061 0.061 0.061 0.061]
 [0.061 0.061 0.061 0.061 0.061]
 [0.061 0.061 0.061 0.061 0.061]
 [0.061 0.061 0.061 0.061 0.061]
 [0.061 0.061 0.061 0.061 0.061]
 [0.061 0.061 0.061 0.061 0.084]]
alpha[:, :, i+1], before projection:
 [[0.982 0.982 0.982 0.983 0.948]
 [0.982 0.982 0.982 0.945 0.947]
 [0.946 0.982 0.982 0.944 0.944]
 [0.943 0.984 0.984 0.941 0.944]
 [0.944 0.982 0.984 0.946 0.944]
 [0.944 0.982 0.984 0.949 0.943]
 [0.947 0.983 0.983 0.983 0.947]]

tau_feas.value: [0.062]

alpha_feas: [[0.982 0.982 0.982 0.983 0.948]
 [0.982 0.982 0.982 0.945 0.947]
 [0.946 0.982 0.982 0.944 0.94


prob.status: optimal

welfare_at_e_t: 18139.97510687325
obj_E_at_e_t: 145.96507173312375
obj_R_at_e_t: 49.800395641871596
obj_I_at_e_t: 18043.810430782

welfare_perturbed_at_e_t: 18139.97510687325
obj_E_perturbed_at_e_t: 145.96507173312375
obj_R_perturbed_at_e_t: 0.0
obj_I_perturbed_at_e_t: 17994.010035140127

tau[:, :, i+1], before projection:
 [[0.01  0.01  0.01  0.01  0.01 ]
 [0.01  0.01  0.01  0.01  0.01 ]
 [0.01  0.01  0.01  0.01  0.01 ]
 [0.01  0.01  0.01  0.01  0.01 ]
 [0.01  0.01  0.01  0.01  0.01 ]
 [0.01  0.01  0.01  0.01  0.01 ]
 [0.01  0.01  0.01  0.01  0.028]]
alpha[:, :, i+1], before projection:
 [[0.982 0.982 0.982 0.983 0.948]
 [0.982 0.982 0.982 0.945 0.947]
 [0.946 0.982 0.982 0.944 0.944]
 [0.943 0.984 0.984 0.941 0.944]
 [0.944 0.982 0.984 0.946 0.944]
 [0.944 0.982 0.984 0.949 0.943]
 [0.947 0.983 0.983 0.983 0.947]]

tau_feas.value: [0.011]

alpha_feas: [[0.982 0.982 0.982 0.983 0.948]
 [0.982 0.982 0.982 0.945 0.947]
 [0.946 0.982 0.982 0.944 0.944]
 [0.943 0.98


prob.status: optimal

welfare_at_e_t: 18139.97510687325
obj_E_at_e_t: 145.96507173312375
obj_R_at_e_t: 0.0
obj_I_at_e_t: 17994.010035140127

welfare_perturbed_at_e_t: 18139.97510687325
obj_E_perturbed_at_e_t: 145.96507173312375
obj_R_perturbed_at_e_t: 12.450098910467899
obj_I_perturbed_at_e_t: 18006.460134050594

tau[:, :, i+1], before projection:
 [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
alpha[:, :, i+1], before projection:
 [[0.982 0.982 0.982 0.983 0.948]
 [0.982 0.982 0.982 0.945 0.947]
 [0.946 0.982 0.982 0.944 0.944]
 [0.943 0.984 0.984 0.941 0.944]
 [0.944 0.982 0.984 0.946 0.944]
 [0.944 0.982 0.984 0.949 0.943]
 [0.947 0.983 0.983 0.983 0.947]]

tau_feas.value: [0.]

alpha_feas: [[0.982 0.982 0.982 0.983 0.948]
 [0.982 0.982 0.982 0.945 0.947]
 [0.946 0.982 0.982 0.944 0.944]
 [0.943 0.984 0.984 0.941 0.944]
 [0.944 0.982 0.984 0.946 0.944]
 [0.944 0.982 0.984 0.949 0.943]
 [0.947 0.983 0.98


prob.status: optimal

welfare_at_e_t: 18139.97510687325
obj_E_at_e_t: 145.96507173312375
obj_R_at_e_t: 0.0
obj_I_at_e_t: 17994.010035140127

welfare_perturbed_at_e_t: 18139.97510687325
obj_E_perturbed_at_e_t: 145.96507173312375
obj_R_perturbed_at_e_t: 0.0
obj_I_perturbed_at_e_t: 17994.010035140127

tau[:, :, i+1], before projection:
 [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
alpha[:, :, i+1], before projection:
 [[0.982 0.982 0.982 0.983 0.948]
 [0.982 0.982 0.982 0.945 0.947]
 [0.946 0.982 0.982 0.944 0.944]
 [0.943 0.984 0.984 0.941 0.944]
 [0.944 0.982 0.984 0.946 0.944]
 [0.944 0.982 0.984 0.949 0.943]
 [0.947 0.983 0.983 0.983 0.947]]

tau_feas.value: [0.]

alpha_feas: [[0.982 0.982 0.982 0.983 0.948]
 [0.982 0.982 0.982 0.945 0.947]
 [0.946 0.982 0.982 0.944 0.944]
 [0.943 0.984 0.984 0.941 0.944]
 [0.944 0.982 0.984 0.946 0.944]
 [0.944 0.982 0.984 0.949 0.943]
 [0.947 0.983 0.983 0.983 0.947]]


prob.status: optimal

welfare_at_e_t: 18139.97510687325
obj_E_at_e_t: 145.96507173312375
obj_R_at_e_t: 0.0
obj_I_at_e_t: 17994.010035140127

welfare_perturbed_at_e_t: 18139.97510687325
obj_E_perturbed_at_e_t: 145.96507173312375
obj_R_perturbed_at_e_t: 0.0
obj_I_perturbed_at_e_t: 17994.010035140127

tau[:, :, i+1], before projection:
 [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
alpha[:, :, i+1], before projection:
 [[0.982 0.982 0.982 0.983 0.948]
 [0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.   ]
 [0.    0.984 0.984 0.941 0.944]
 [0.944 0.982 0.984 0.946 0.944]
 [0.944 0.982 0.984 0.949 0.943]
 [0.947 0.983 0.983 0.983 0.947]]

tau_feas.value: [0.]

alpha_feas: [[0.982 0.982 0.982 0.983 0.948]
 [0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.   ]
 [0.    0.984 0.984 0.941 0.944]
 [0.944 0.982 0.984 0.946 0.944]
 [0.944 0.982 0.984 0.949 0.943]
 [0.947 0.983 0.983 0.983 0.947]]

In [150]:
e, t = 2, 4

print("welfares[e, t, :]:", welfares[e, t, :])
print()
print("tau[e, t, :]:", tau[e, t, :])
print()
print("alpha[e, t, :]:", alpha[e, t, :])

welfares[e, t, :]: [42025.54695876 42025.54695876 42025.54695876 42025.54695876
 42025.54695876 42025.54695876 42025.54695876 42025.54695876
 42025.54695876 42025.54695876 42025.54695876 42025.54695876
 42025.54695876     0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
   

In [82]:
# # y_agg_el_at_e_t
# print("y_agg_el_ex:", y_agg_el_ex)

# e_temp = 1
# sum([demand_array[od, g] for od in edge_to_od_dict[e_temp] for g in el_indices])

In [ ]:
# y_DBCP_init_at_e_t

# assert y_DBCP_init_at_e_t == extract_y_at_e_t(y_DBCP_init, 6, 4)

In [ ]:
# for e in range(num_edges):
#     print("Edge:", e)
    
#     assert equals(obj_DBCP_init_together[(e, 'E')], obj_DBCP_init[(e, 'E')])
#     assert equals(obj_DBCP_init_together[(e, 'R')], obj_DBCP_init[(e, 'R')])
#     assert equals(obj_DBCP_init_together[(e, 'I')], obj_DBCP_init[(e, 'I')])
    

# Old Code

In [ ]:
# list_1 = [1, 2, 3]
# list_2 = [2, 3, 4]

# [x for x in list_1 if x not in list_2]


In [ ]:
edge_to_od_dict

In [ ]:
# init_DBCP_key_list_comp = [(od, g, e, k, t) for e in range(num_edges) for t in range(T) \
#                               for k in [0, 1] for g in (el_indices + in_indices) for od in edge_to_od_dict[e]]

# list(init_DBCP_flows.keys()) == init_DBCP_key_list_comp

# keys_missing_init_DBCP_flows = [key for key in list(init_DBCP_flows.keys()) \
#                                 if key not in init_DBCP_key_list_comp]
# keys_extra_init_DBCP_flows = [key for key in init_DBCP_key_list_comp \
#                                 if key not in list(init_DBCP_flows.keys())]

# print("keys_missing_init_DBCP_flows:", keys_missing_init_DBCP_flows)
# print("\n")
# print("keys_extra_init_DBCP_flows:", keys_extra_init_DBCP_flows)

In [ ]:
y_in_el_total_init_DBCP = {}
x_init_DBCP = {}

for e in range(num_edges):
    for t in range(T):
        for k in [0, 1]:
            y_in_el_total_init_DBCP[(e, k, t, "in")] \
                = sum(init_DBCP_flows[(od, g, e, k, t)] for od in edge_to_od_dict[e] for g in in_indices)
            y_in_el_total_init_DBCP[(e, k, t, "el")] \
                = sum(init_DBCP_flows[(od, g, e, k, t)] for od in edge_to_od_dict[e] for g in el_indices)
            
            x_init_DBCP[(e, k, t)] = y_in_el_total_init_DBCP[(e, k, t, "in")] \
                                        + y_in_el_total_init_DBCP[(e, k, t, "el")]


In [ ]:
y_init_DBCP

In [ ]:
y_in_el_total_init_DBCP

In [ ]:
x_init_DBCP

In [ ]:
# lambda_R = 1

travel_times_DBCP = {}

## coeff_input: const, slope, x-coordinate of transition point
# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))

for e in range(num_edges):
    for t in range(T):
        travel_times_DBCP[(e, 0, t)] \
            = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 0, t)] - coeff_input[2, e], 0.0)
        travel_times_DBCP[(e, 1, t)] \
            = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 1, t)]/num_gp_lanes - coeff_input[2, e], 0.0)

# INCOMPLETE:
        
edge_demand_DBCP = {}
avg_travel_time_DBCP = {}
percent_on_express_DBCP = {}
obj_E_DBCP = np.zeros(num_edges)
obj_I_DBCP = np.zeros(num_edges)
obj_R_DBCP = np.zeros(num_edges)
obj_DBCP = np.zeros(num_edges)

for e in range(num_edges):
    avg_travel_time_DBCP[e, 'el'] = 0.0
    avg_travel_time_DBCP[e, 'in'] = 0.0
    avg_travel_time_DBCP[e, 'ex'] = 0.0
    avg_travel_time_DBCP[e, 'gp'] = 0.0
    
    percent_on_express_DBCP[e, 'el'] = 0.0
    percent_on_express_DBCP[e, 'in'] = 0.0
    percent_on_express_DBCP[e, 'all'] = 0.0
    
    obj_E_DBCP[e] = 0.0
    obj_I_DBCP[e] = 0.0
    obj_R_DBCP[e] = 0.0
    obj_DBCP[e] = 0.0
    
    for t in range(T):
        edge_demand_DBCP[e, t, 'el'] = sum(argmin_y_in_el_total[(e, k, t, 'el')] for k in range(2))
        edge_demand_DBCP[e, t, 'in'] = sum(argmin_y_in_el_total[(e, k, t, 'in')] for k in range(2))
        
    percent_on_express_DBCP[e, 'el'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'el'] for t in range(T))
    percent_on_express_DBCP[e, 'in'] += sum(argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'in'] for t in range(T))
    percent_on_express_DBCP[e, 'all'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'el'] + edge_demand_DBCP[e, t, 'in'] for t in range(T))
    
    avg_travel_time_DBCP[e, 'el'] += sum(argmin_y_in_el_total[(e, k, t, 'el')] * travel_times_DBCP[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'el'] for t in range(T))    
    avg_travel_time_DBCP[e, 'in'] += sum(argmin_y_in_el_total[(e, k, t, 'in')] * travel_times_DBCP[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'in'] for t in range(T))    
#     avg_travel_time_DBCP[e, 'ex'] += sum( (argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')]) * travel_times_DBCP[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T))
#     avg_travel_time_DBCP[e, 'gp'] += sum( (argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')]) * travel_times_DBCP[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')] for t in range(T))
    avg_travel_time_DBCP[e, 'ex'] += sum(travel_times_DBCP[(e, 0, t)] for t in range(T)) / T 
    avg_travel_time_DBCP[e, 'gp'] += sum(travel_times_DBCP[(e, 1, t)] for t in range(T)) / T 
    
    obj_E_DBCP[e] = sum( y_init_DBCP[(od, g, e, 0, t)] * VoT_array[od, g, t] * travel_times_DBCP[e, 0, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
                    + sum( y_init_DBCP[(od, g, e, 0, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) * (1 - init_alpha[e, t]) \
                    + sum( y_init_DBCP[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times_DBCP[e, 1, t] \
                          for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 
    
#                     + sum( y_init_DBCP[(od, g, e, 1, t)] * (VoT_array[od, g, t] * travel_times_DBCP[e, 0, t] + argmin_tau[e, t]) \
#                         for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \

    obj_I_DBCP[e] = sum( y_init_DBCP[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times_DBCP[e, 0, t] + argmin_tau[e, t]) \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( y_init_DBCP[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times_DBCP[e, 1, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

    obj_R_DBCP[e] = sum( y_init_DBCP[(od, g, e, 0, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( y_init_DBCP[(od, g, e, 0, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) * (1 - init_alpha[e, t])

    obj_DBCP[e] = lambda_E * obj_E_DBCP[e] - lambda_R * obj_R_DBCP[e] + lambda_I * obj_I_DBCP[e]


# welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, argmin_tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input)

# avg_travel_time_DBCP
# percent_on_express_DBCP
# obj_R_DBCP

sum(obj_DBCP)

In [ ]:
# column_names = []
# column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
# column_names += ["alpha (t=" + str(t+1) + ")" for t in range(T) ]

# row_names = ["e=" + str(k+1) for k in range(num_edges) ]

# df_inits_save = pd.DataFrame(init_tau_alpha_array, index=row_names, columns=column_names)

# # df_inits_save

# # random_string = ""
# # for idx_rand_str in range(10):
# #     random_string += str(np.random.randint(1, 9))

# directory_to_save = "../data/old___opt_tolls_subsidies_metrics/"
# random_filename = "inits___" + random_string + '.csv'

# df_inits_save.to_csv(directory_to_save + random_filename)

In [ ]:
print("first(welfares):", welfares[0])
print("min(welfares):", min(welfares))
print("max(welfares):", max(welfares))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

## Test:

## <font color='red'>Colored Font Titles</font> 

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


## Linear Approximation for Latency Function:

In [ ]:
# Variables:
v = cp.Variable(1)
            
# Objective:
func = v - 1 + cp.square(cp.maximum(v-1, 0))
objective = cp.Minimize(func)

# Constraints:
constraints = [-3.0 <= v, v <= 3.0]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("v.value:", v.value)
